In [131]:
import pyomo.environ as pyo


## EX1

In [132]:
# Criação do modelo
model = pyo.ConcreteModel("Racao")

In [133]:
# Tipos de ração:
model.R = pyo.Set(initialize=["Tobi", "Rex"])

# Tipos de material
model.M = pyo.Set(initialize=["Cereal", "Carne"])

In [134]:
# ------------------------------
# Parâmetros
# ------------------------------
# Custos material
material = {"Cereal":1, "Carne":4}
model.c_material = pyo.Param(model.M,initialize=material)

# Qtd material por pacote
qtd_material = {("Tobi","Cereal"):5, ("Rex","Cereal"):2,("Tobi","Carne"):1, ("Rex","Carne"):4}
model.q_material = pyo.Param(model.R,model.M,initialize=qtd_material)

# Material disponível

estoque = {"Cereal":30000, "Carne":10000}
model.e_material = pyo.Param(model.M,initialize=estoque)

# Custos pacote
valor_venda = {"Tobi":20, "Rex":30}
model.v_pacote = pyo.Param(model.R,initialize=valor_venda)



In [135]:
# ------------------------------
# Variáveis de Decisão
# ------------------------------
# quantidade da ração tobi e max
model.x = pyo.Var(model.R,domain=pyo.NonNegativeIntegers)

In [136]:
# ------------------------------
# Função Objetivo
# ------------------------------
def objective_rule(model):
    # Valor pacote
    valor_pacote = sum(
        model.x[i] * model.v_pacote[i]
        for i in model.R
    )

    # Custo material
    custo = sum(
        model.x[i] * model.c_material[m] * model.q_material[i,m]
        for i in model.R
        for m in model.M
    )

    return valor_pacote + custo


model.obj = pyo.Objective(rule=objective_rule, sense=pyo.maximize)

In [137]:
# ------------------------------
# Restrições
# ------------------------------

# 1. Limite Estoque
def estoque(model, m):
    return sum(model.x[i] * model.q_material[i,m] for i in model.R) <= model.e_material[m]


model.CapRef = pyo.Constraint(model.M, rule=estoque)


In [138]:
# ------------------------------
# Escrita do Modelo em Arquivo
# ------------------------------
model.write("ex1.lp", io_options={"symbolic_solver_labels": True})

# ------------------------------
# Resolução
# ------------------------------
solver = pyo.SolverFactory("appsi_highs")
results = solver.solve(model, tee=True)

# Exibindo resultados
print("\nStatus do solver:", results.solver.status)
print("Condição de terminação:", results.solver.termination_condition)
valor_obj = pyo.value(model.obj)
formatted = f"{valor_obj:,.2f}"
formatted = formatted.replace(",", "X").replace(".", ",").replace("X", ".")
print("Valor da Função Objetivo: R$", formatted)



MIP  has 2 rows; 2 cols; 4 nonzeros; 2 integer variables (0 binary)
Coefficient ranges:
  Matrix [1e+00, 5e+00]
  Cost   [3e+01, 5e+01]
  Bound  [0e+00, 0e+00]
  RHS    [1e+04, 3e+04]
Presolving model
2 rows, 2 cols, 4 nonzeros  0s
2 rows, 2 cols, 4 nonzeros  0s
Objective function is integral with scale 1

Solving MIP model with:
   2 rows
   2 cols (0 binary, 2 integer, 0 implied int., 0 continuous)
   4 nonzeros

Src: B => Branching; C => Central rounding; F => Feasibility pump; H => Heuristic; L => Sub-MIP;
     P => Empty MIP; R => Randomized rounding; S => Solve LP; T => Evaluate node; U => Unbounded;
     z => Trivial zero; l => Trivial lower; u => Trivial upper; p => Trivial point; X => User solution

        Nodes      |    B&B Tree     |            Objective Bounds              |  Dynamic Constraints |       Work      
Src  Proc. InQueue |  Leaves   Expl. | BestBound       BestSol              Gap |   Cuts   InLp Confl. | LpIters     Time

 z       0       0         0   0.00% 

In [139]:
for i in model.R:    
    val = pyo.value(model.x[i])
    if val is not None and val > 1e-6:
        print(f"  Quantidade da ração {i} : {val:.2f}")

  Quantidade da ração Tobi : 5555.00
  Quantidade da ração Rex : 1111.00


## EX2

In [140]:
# Criação do modelo
model = pyo.ConcreteModel("")

In [141]:
# Tipos produto:
model.P = pyo.Set(initialize=["1", "2"])

In [142]:
# ------------------------------
# Parâmetros
# ------------------------------

# lucro produto material
luc = {"1":5, "2":6}
model.lucro = pyo.Param(model.P,initialize=luc)

# Custos material
material = {"1":1, "2":2}
model.mat = pyo.Param(model.P,initialize=material)

# Mão de obra
mao_obra = {"1":1, "2":1}
model.mao_de_obra = pyo.Param(model.P,initialize=mao_obra)

# Tempo maquinario
maq = {"1":7, "2":4}
model.temp_maq = pyo.Param(model.P,initialize=maq)

# Material disponível
model.e_material = pyo.Param(initialize=14)

#Mão de obra disponivel
model.e_mao_obra = pyo.Param(initialize=9)

#Tempo Maquinario disponivel
model.e_temp_maq = pyo.Param(initialize=56)

In [143]:
# ------------------------------
# Variáveis de Decisão
# ------------------------------
# quantidade da ração tobi e max
model.x = pyo.Var(model.P,domain=pyo.NonNegativeIntegers)

In [144]:
# ------------------------------
# Função Objetivo
# ------------------------------
def objective_rule(model):
    # Lucro por produto
    valor_pacote = sum(
        model.x[i] * model.lucro[i]
        for i in model.P
    )

    return valor_pacote


model.obj = pyo.Objective(rule=objective_rule, sense=pyo.maximize)

In [145]:
# ------------------------------
# Restrições
# ------------------------------

# 1. Limite Materia
def meteria(model):
    return sum(model.x[i] * model.mat[i] for i in model.P) <= model.e_material

model.re_materia= pyo.Constraint(rule=meteria)

# 2. Limite Maõ de Obra
def mao_obra(model):
    return sum(model.x[i] * model.mao_de_obra[i] for i in model.P) <= model.e_mao_obra

model.re_mao_obra = pyo.Constraint(rule=mao_obra)

# 3. Tempo maquinario
def maquinario(model):
    return sum(model.x[i] * model.temp_maq[i] for i in model.P) <= model.e_temp_maq

model.re_maq = pyo.Constraint(rule=maquinario)


In [146]:
# ------------------------------
# Escrita do Modelo em Arquivo
# ------------------------------
model.write("ex2.lp", io_options={"symbolic_solver_labels": True})

# ------------------------------
# Resolução
# ------------------------------
solver = pyo.SolverFactory("appsi_highs")
results = solver.solve(model, tee=True)

# Exibindo resultados
print("\nStatus do solver:", results.solver.status)
print("Condição de terminação:", results.solver.termination_condition)
valor_obj = pyo.value(model.obj)
formatted = f"{valor_obj:,.2f}"
formatted = formatted.replace(",", "X").replace(".", ",").replace("X", ".")
print("Valor da Função Objetivo: R$", formatted)



MIP  has 3 rows; 2 cols; 6 nonzeros; 2 integer variables (0 binary)
Coefficient ranges:
  Matrix [1e+00, 7e+00]
  Cost   [5e+00, 6e+00]
  Bound  [0e+00, 0e+00]
  RHS    [9e+00, 6e+01]
Presolving model
3 rows, 2 cols, 6 nonzeros  0s
3 rows, 2 cols, 6 nonzeros  0s
Objective function is integral with scale 1

Solving MIP model with:
   3 rows
   2 cols (0 binary, 2 integer, 0 implied int., 0 continuous)
   6 nonzeros

Src: B => Branching; C => Central rounding; F => Feasibility pump; H => Heuristic; L => Sub-MIP;
     P => Empty MIP; R => Randomized rounding; S => Solve LP; T => Evaluate node; U => Unbounded;
     z => Trivial zero; l => Trivial lower; u => Trivial upper; p => Trivial point; X => User solution

        Nodes      |    B&B Tree     |            Objective Bounds              |  Dynamic Constraints |       Work      
Src  Proc. InQueue |  Leaves   Expl. | BestBound       BestSol              Gap |   Cuts   InLp Confl. | LpIters     Time

 z       0       0         0   0.00% 

In [147]:
for i in model.P:    
    val = pyo.value(model.x[i])
    if val is not None and val > 1e-6:
        print(f"  Quantidade do produto {i} : {val:.2f}")

  Quantidade do produto 1 : 4.00
  Quantidade do produto 2 : 5.00


## EX3

In [148]:
# Criação do modelo
model = pyo.ConcreteModel("Passarinho")

In [149]:
# Tipos produto:
model.P = pyo.Set(initialize=["forte", "light"])

# Tipos material:
model.M = pyo.Set(initialize=["1", "2"])

In [150]:
# ------------------------------
# Parâmetros
# ------------------------------


# Custos 
custo = {"forte":6, "light":10}
model.c = pyo.Param(model.P,initialize=custo)

# Qtd mão de obra
mao_obra = {"forte":1, "light":2}
model.mao_de_obra = pyo.Param(model.P,initialize=mao_obra)

# Limite producao
limt = {"forte":(0,5), "light":(0,6)}
# model.limite = pyo.Param(model.P,initialize=limt)

# Material
qtd_material = {("forte","1"):3, ("light","1"):5,("forte","2"):5, ("light","2"):4}
model.q_material = pyo.Param(model.P,model.M,initialize=qtd_material)

#utilização material
ut = {"1":15, "2":20}
model.min_material = pyo.Param(model.M,initialize=ut)

# diferença material
model.diff_material = pyo.Param(initialize=2)

In [151]:
# ------------------------------
# Variáveis de Decisão
# ------------------------------
# quantidade da ração tobi e max
model.x = pyo.Var(model.P,domain=pyo.NonNegativeIntegers,bounds = limt)

In [152]:
# ------------------------------
# Função Objetivo
# ------------------------------
def objective_rule(model):
    fo = sum(
        model.x[i] * model.c[i]
        for i in model.P
    )

    return fo

model.obj = pyo.Objective(rule=objective_rule, sense=pyo.minimize)

In [153]:
# ------------------------------
# Restrições
# ------------------------------

# 1. Diff quantidade de produtos - upper
def up_meteria(model):
    return model.x['light'] - model.x['forte'] <= 2

model.re_materia_up= pyo.Constraint(rule=up_meteria)

#1. Diff quantidade de produtos - lower
def lw_meteria(model):
    return model.x['light'] - model.x['forte']  >= 0

model.re_materia_lw= pyo.Constraint(rule=lw_meteria)

# 2. Material
def disponibilidade_material(model,m):
    return sum(model.x[i] * model.q_material[i,m] for i in model.P) >= model.min_material[m]

model.re_material = pyo.Constraint(model.M,rule=disponibilidade_material)



In [154]:
# ------------------------------
# Escrita do Modelo em Arquivo
# ------------------------------
model.write("ex3.lp", io_options={"symbolic_solver_labels": True})

# ------------------------------
# Resolução
# ------------------------------
solver = pyo.SolverFactory("appsi_highs")
results = solver.solve(model, tee=True)

# Exibindo resultados
print("\nStatus do solver:", results.solver.status)
print("Condição de terminação:", results.solver.termination_condition)
valor_obj = pyo.value(model.obj)
formatted = f"{valor_obj:,.2f}"
formatted = formatted.replace(",", "X").replace(".", ",").replace("X", ".")
print("Valor da Função Objetivo: R$", formatted)



MIP  has 4 rows; 2 cols; 8 nonzeros; 2 integer variables (0 binary)
Coefficient ranges:
  Matrix [1e+00, 5e+00]
  Cost   [6e+00, 1e+01]
  Bound  [5e+00, 6e+00]
  RHS    [2e+00, 2e+01]
Presolving model
4 rows, 2 cols, 8 nonzeros  0s
3 rows, 2 cols, 6 nonzeros  0s
3 rows, 2 cols, 6 nonzeros  0s
Objective function is integral with scale 0.5

Solving MIP model with:
   3 rows
   2 cols (0 binary, 2 integer, 0 implied int., 0 continuous)
   6 nonzeros

Src: B => Branching; C => Central rounding; F => Feasibility pump; H => Heuristic; L => Sub-MIP;
     P => Empty MIP; R => Randomized rounding; S => Solve LP; T => Evaluate node; U => Unbounded;
     z => Trivial zero; l => Trivial lower; u => Trivial upper; p => Trivial point; X => User solution

        Nodes      |    B&B Tree     |            Objective Bounds              |  Dynamic Constraints |       Work      
Src  Proc. InQueue |  Leaves   Expl. | BestBound       BestSol              Gap |   Cuts   InLp Confl. | LpIters     Time

 u  

In [155]:
for i in model.P:    
    val = pyo.value(model.x[i])
    if val is not None and val > 1e-6:
        print(f"  Quantidade do produto {i} : {val:.2f}")

  Quantidade do produto forte : 2.00
  Quantidade do produto light : 3.00


## Ex4

In [156]:
# Criação do modelo
model = pyo.ConcreteModel()

In [157]:

# Conjuntos
model.P = pyo.Set(initialize=['Arlington', 'Marilandy', 'Gristedes'])  # Modelos de carros
model.S = pyo.Set(initialize=['Injecao', 'Fundicao', 'Usinagem', 'Estamparia', 'Acabamento'])  # Setores

In [158]:
# ------------------------------
# Parâmetros
# ------------------------------
lucro = {'Arlington': 2500, 'Marilandy': 3000, 'Gristedes': 2800}
model.lucro = pyo.Param(model.P,initialize=lucro)

demanda_min = {'Arlington': 50, 'Marilandy': 30, 'Gristedes': 30}
model.demanda_min = pyo.Param(model.P,initialize=demanda_min)

tempo_operacao = {
    ('Injecao', 'Arlington'): 3, ('Injecao', 'Marilandy'): 4, ('Injecao', 'Gristedes'): 3,
    ('Fundicao', 'Arlington'): 5, ('Fundicao', 'Marilandy'): 5, ('Fundicao', 'Gristedes'): 4,
    ('Usinagem', 'Arlington'): 2, ('Usinagem', 'Marilandy'): 4, ('Usinagem', 'Gristedes'): 4,
    ('Estamparia', 'Arlington'): 4, ('Estamparia', 'Marilandy'): 5, ('Estamparia', 'Gristedes'): 5,
    ('Acabamento', 'Arlington'): 2, ('Acabamento', 'Marilandy'): 3, ('Acabamento', 'Gristedes'): 3
}
model.tempo_operacao = pyo.Param(model.S, model.P, initialize=tempo_operacao)

maquinas_disponiveis = {'Injecao': 6, 'Fundicao': 8, 'Usinagem': 5, 'Estamparia': 8, 'Acabamento': 5}
model.maquinas_disponiveis = pyo.Param(model.S, initialize=maquinas_disponiveis)

tempo_semanal_por_maquina = 4800  # minutos por semana
model.tempo_semanal_por_maquina = pyo.Param(initialize=tempo_semanal_por_maquina)

In [159]:
# ------------------------------
# Variáveis de Decisão
# ------------------------------
model.x = pyo.Var(model.P, domain=pyo.NonNegativeReals)  # Quantidade produzida de cada modelo

In [160]:
# ------------------------------
# Função Objetivo
# ------------------------------
def lucro_total(model):
    return sum(lucro[j] * model.x[j] for j in model.P)
model.objetivo = pyo.Objective(rule=lucro_total, sense=pyo.maximize)

In [161]:

# ------------------------------
# Restrições
# ------------------------------


# 1. Restrições de capacidade por setor
def restricao_capacidade(model, i):
    return sum(tempo_operacao[i, j] * model.x[j] for j in model.P) <= maquinas_disponiveis[i] * tempo_semanal_por_maquina
model.capacidade = pyo.Constraint(model.S, rule=restricao_capacidade)

# 2. Restrições de demanda mínima
def restricao_demanda(model, j):
    return model.x[j] >= demanda_min[j]
model.demanda = pyo.Constraint(model.P, rule=restricao_demanda)

In [162]:
# ------------------------------
# Escrita do Modelo em Arquivo
# ------------------------------
model.write("ex4.lp", io_options={"symbolic_solver_labels": True})

('ex4.lp', 2676203678112)

In [163]:
# ------------------------------
# Resolução
# ------------------------------
solver = pyo.SolverFactory("appsi_highs")
results = solver.solve(model, tee=True)

# Exibindo resultados
print("\nStatus do solver:", results.solver.status)
print("Condição de terminação:", results.solver.termination_condition)
valor_obj = pyo.value(model.objetivo)
formatted = f"{valor_obj:,.2f}"
formatted = formatted.replace(",", "X").replace(".", ",").replace("X", ".")
print("Valor da Função Objetivo: R$", formatted)



LP   has 8 rows; 3 cols; 18 nonzeros
Coefficient ranges:
  Matrix [1e+00, 5e+00]
  Cost   [2e+03, 3e+03]
  Bound  [0e+00, 0e+00]
  RHS    [3e+01, 4e+04]
Presolving model
5 rows, 3 cols, 15 nonzeros  0s
5 rows, 3 cols, 15 nonzeros  0s
Presolve : Reductions: rows 5(-3); columns 3(-0); elements 15(-3)
Solving the presolved LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0    -8.2999936572e+03 Ph1: 5(56); Du: 3(8299.99) 0s
          2    -2.2075000000e+07 Pr: 0(0) 0s
Solving the original LP from the solution after postsolve
Model status        : Optimal
Simplex   iterations: 2
Objective value     :  2.2075000000e+07
Relative P-D gap    :  0.0000000000e+00
HiGHS run time      :          0.00

Status do solver: ok
Condição de terminação: optimal
Valor da Função Objetivo: R$ 22.075.000,00


In [164]:
for i in model.P:    
    val = pyo.value(model.x[i])
    if val is not None and val > 1e-6:
        print(f"  Quantidade do produto {i} : {val:.2f}")

  Quantidade do produto Arlington : 4790.00
  Quantidade do produto Marilandy : 30.00
  Quantidade do produto Gristedes : 3575.00


## Ex5 

In [165]:
# Criação do modelo
model = pyo.ConcreteModel()

In [166]:

# Conjuntos
model.P = pyo.Set(initialize=['Cerveja', 'Refrigerante']) 

model.S_cerveja = pyo.Set(initialize=['Malte', 'Mosto', 'Fermentacao', 'Processamento', 'Envase'])  
model.S_refrigerante = pyo.Set(initialize=['Xarope_Simples', 'Xarope_Composto', 'Diluicao', 'Carbonatacao', 'Envase'])  

In [167]:
# ------------------------------
# Parâmetros
# ------------------------------


tempo_cerveja = {'Malte': 2, 'Mosto': 4, 'Fermentacao': 3, 'Processamento': 4, 'Envase': 5}
model.tempo_cerveja = pyo.Param(model.S_cerveja, initialize=tempo_cerveja)

tempo_refrigerante = {'Xarope_Simples': 1, 'Xarope_Composto': 3, 'Diluicao': 4, 'Carbonatacao': 5, 'Envase': 2}
model.tempo_refrigerante = pyo.Param(model.S_refrigerante, initialize=tempo_refrigerante)

maquinas_cerveja = {'Malte': 6, 'Mosto': 12, 'Fermentacao': 10, 'Processamento': 12, 'Envase': 13}
model.maquinas_cerveja = pyo.Param(model.S_cerveja, initialize=maquinas_cerveja)

maquinas_refrigerante = {'Xarope_Simples': 6, 'Xarope_Composto': 7, 'Diluicao': 8, 'Carbonatacao': 10, 'Envase': 5}
model.maquinas_refrigerante = pyo.Param(model.S_refrigerante, initialize=maquinas_refrigerante)

tempo_disponivel = 8 * 20 * 60  # 20 dias úteis de 8 horas por dia, em minutos
model.tempo_disponivel = pyo.Param(initialize=tempo_disponivel)

margem_contribuicao = {'Cerveja': 0.5, 'Refrigerante': 0.4}
model.margem_contribuicao = pyo.Param(model.P, initialize=margem_contribuicao)

demanda_maxima = 42000  # Demanda máxima total por mês em litros
model.demanda_maxima = pyo.Param(initialize=demanda_maxima)


In [168]:

# ------------------------------
# Variáveis de Decisão
# ------------------------------
model.x = pyo.Var(model.P, domain=pyo.NonNegativeReals)  # Quantidade produzida de cada modelo


In [169]:

# ------------------------------
# Função Objetivo
# ------------------------------
def MC_total(model):
    return sum(margem_contribuicao[j] * model.x[j] for j in model.P)
model.objetivo = pyo.Objective(rule=MC_total, sense=pyo.maximize)


In [170]:
# ------------------------------
# Restrições
# ------------------------------

# 1. Restrições de capacidade cerveja
def restricao_capacidade_cerveja(model, i):

    return model.tempo_cerveja[i] * model.x['Cerveja'] <= model.maquinas_cerveja[i] * model.tempo_disponivel
model.capacidade_cerv = pyo.Constraint(model.S_cerveja, rule=restricao_capacidade_cerveja)

# 2. Restrições de capacidade refirgerante
def restricao_capacidade_refri(model, i):

    return model.tempo_refrigerante[i] * model.x['Refrigerante'] <= model.maquinas_refrigerante[i] * model.tempo_disponivel
model.capacidade_refri = pyo.Constraint(model.S_refrigerante, rule=restricao_capacidade_refri)

# 3. Restrições de demanda máxima
def restricao_demanda(model):
    return sum(model.x[j] for j in model.P) <= model.demanda_maxima
model.demanda = pyo.Constraint(rule=restricao_demanda)

In [171]:
# ------------------------------
# Escrita do Modelo em Arquivo
# ------------------------------
model.write("ex5.lp", io_options={"symbolic_solver_labels": True})

('ex5.lp', 2676203634800)

In [172]:
# ------------------------------
# Resolução
# ------------------------------
solver = pyo.SolverFactory("appsi_highs")
results = solver.solve(model, tee=True)

# Exibindo resultados
print("\nStatus do solver:", results.solver.status)
print("Condição de terminação:", results.solver.termination_condition)
valor_obj = pyo.value(model.objetivo)
formatted = f"{valor_obj:,.2f}"
formatted = formatted.replace(",", "X").replace(".", ",").replace("X", ".")
print("Valor da Função Objetivo: R$", formatted)



LP   has 11 rows; 2 cols; 12 nonzeros
Coefficient ranges:
  Matrix [1e+00, 5e+00]
  Cost   [4e-01, 5e-01]
  Bound  [0e+00, 0e+00]
  RHS    [4e+04, 1e+05]
Presolving model
1 rows, 2 cols, 2 nonzeros  0s
1 rows, 2 cols, 2 nonzeros  0s
Presolve : Reductions: rows 1(-10); columns 2(-0); elements 2(-10)
Solving the presolved LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0     0.0000000000e+00 Ph1: 0(0) 0s
          1    -1.9296000000e+04 Pr: 0(0) 0s
Solving the original LP from the solution after postsolve
Model status        : Optimal
Simplex   iterations: 1
Objective value     :  1.9296000000e+04
Relative P-D gap    :  0.0000000000e+00
HiGHS run time      :          0.00

Status do solver: ok
Condição de terminação: optimal
Valor da Função Objetivo: R$ 19.296,00


In [173]:
for i in model.P:    
    val = pyo.value(model.x[i])
    if val is not None and val > 1e-6:
        print(f"  Quantidade do produto {i} : {val:.2f}")

  Quantidade do produto Cerveja : 24960.00
  Quantidade do produto Refrigerante : 17040.00


## EX6

In [174]:
# Criação do modelo
model = pyo.ConcreteModel()

In [175]:

# Conjuntos
model.P = pyo.Set(initialize=['Geladeira', 'Freezer', 'Fogao', 'LavaLoucas', 'Microondas'])  
model.S = pyo.Set(initialize=['Prensagem', 'Pintura', 'Liquefacao', 'Montagem', 'Embalagem']) 

In [176]:
# ------------------------------
# Parâmetros
# ------------------------------
# Horas-máquina necessárias por produto e setor
HM = {('Geladeira', 'Prensagem'): 0.2, ('Geladeira', 'Pintura'): 0.2, ('Geladeira', 'Liquefacao'): 0.4, ('Geladeira', 'Montagem'): 0.2, ('Geladeira', 'Embalagem'): 0.1,
      ('Freezer', 'Prensagem'): 0.2, ('Freezer', 'Pintura'): 0.3, ('Freezer', 'Liquefacao'): 0.3, ('Freezer', 'Montagem'): 0.4, ('Freezer', 'Embalagem'): 0.2,
      ('Fogao', 'Prensagem'): 0.4, ('Fogao', 'Pintura'): 0.3, ('Fogao', 'Liquefacao'): 0.3, ('Fogao', 'Montagem'): 0.4, ('Fogao', 'Embalagem'): 0.2,
      ('LavaLoucas', 'Prensagem'): 0.4, ('LavaLoucas', 'Pintura'): 0.3, ('LavaLoucas', 'Liquefacao'): 0.3, ('LavaLoucas', 'Montagem'): 0.4, ('LavaLoucas', 'Embalagem'): 0.2,
      ('Microondas', 'Prensagem'): 0.3, ('Microondas', 'Pintura'): 0.2, ('Microondas', 'Liquefacao'): 0.2, ('Microondas', 'Montagem'): 0.4, ('Microondas', 'Embalagem'): 0.3}
model.HM = pyo.Param(model.P, model.S, initialize=HM)
# Horas-máquina necessárias por produto e setor
HH = {('Geladeira', 'Prensagem'): 0.5, ('Geladeira', 'Pintura'): 0.3, ('Geladeira', 'Liquefacao'): 0.5, ('Geladeira', 'Montagem'): 0.6, ('Geladeira', 'Embalagem'): 0.4,
      ('Freezer', 'Prensagem'): 0.4, ('Freezer', 'Pintura'): 0.4, ('Freezer', 'Liquefacao'): 0.5, ('Freezer', 'Montagem'): 0.5, ('Freezer', 'Embalagem'): 0.4,
      ('Fogao', 'Prensagem'): 0.5, ('Fogao', 'Pintura'): 0.4, ('Fogao', 'Liquefacao'): 0.3, ('Fogao', 'Montagem'): 0.4, ('Fogao', 'Embalagem'): 0.4,
      ('LavaLoucas', 'Prensagem'): 0.4, ('LavaLoucas', 'Pintura'): 0.4, ('LavaLoucas', 'Liquefacao'): 0.4, ('LavaLoucas', 'Montagem'): 0.5, ('LavaLoucas', 'Embalagem'): 0.3,
      ('Microondas', 'Prensagem'): 0.2, ('Microondas', 'Pintura'): 0.3, ('Microondas', 'Liquefacao'): 0.3, ('Microondas', 'Montagem'): 0.6, ('Microondas', 'Embalagem'): 0.2}
model.HH = pyo.Param(model.P, model.S, initialize=HH)
 # Horas-máquina disponíveis por setor
T = {'Prensagem': 400, 'Pintura': 350, 'Liquefacao': 250, 'Montagem': 200, 'Embalagem': 200} 
model.T = pyo.Param(model.S, initialize=T)
# Funcionários disponíveis por setor
H = {'Prensagem': 120, 'Pintura': 10, 'Liquefacao': 8, 'Montagem': 10, 'Embalagem': 8}  
model.H = pyo.Param(model.S, initialize=H)
# Horas de trabalho por dia
model.h = pyo.Param(initialize=8)
# Dias de trabalho por semana
model.d = pyo.Param(initialize=5)  
# Lucro unitário por produto
L = {'Geladeira': 52, 'Freezer': 37, 'Fogao': 35, 'LavaLoucas': 40, 'Microondas': 29} 
model.L = pyo.Param(model.P, initialize=L)
# Demanda mínima
Cm = {'Geladeira': 200, 'Freezer': 50, 'Fogao': 50, 'LavaLoucas': 50, 'Microondas': 40}  
model.Cm = pyo.Param(model.P, initialize=Cm)
# Demanda máxima
CM = {'Geladeira': 1000, 'Freezer': 800, 'Fogao': 500, 'LavaLoucas': 500, 'Microondas': 200}  
model.CM = pyo.Param(model.P, initialize=CM)

In [177]:

# ------------------------------
# Variáveis de Decisão
# ------------------------------
model.x = pyo.Var(model.P, domain=pyo.NonNegativeReals)  # Quantidade produzida de cada modelo


In [178]:

# ------------------------------
# Função Objetivo
# ------------------------------
def lucro_total(model):
    return sum(model.L[j] * model.x[j] for j in model.P)
model.objetivo = pyo.Objective(rule=lucro_total, sense=pyo.maximize)


In [179]:
# ------------------------------
# Restrições
# ------------------------------


# Restrições de disponibilidade de máquinas
def restricao_maquinas(model, j):
    return sum(HM[i, j] * model.x[i] for i in model.P) <= T[j]
model.restricoes_maquinas = pyo.Constraint(model.S, rule=restricao_maquinas)

# Restrições de disponibilidade de mão de obra
def restricao_mao_de_obra(model, j):
    return sum(HH[i, j] * model.x[i] for i in model.P) <= H[j] * model.h * model.d
model.restricoes_mao_de_obra = pyo.Constraint(model.S, rule=restricao_mao_de_obra)

# Restrições de capacidade mínima
def restricao_minima(model, i):
    return model.x[i] >= Cm[i]
model.restricoes_minimas = pyo.Constraint(model.P, rule=restricao_minima)

# Restrições de capacidade máxima
def restricao_maxima(model, i):
    return model.x[i] <= CM[i]
model.restricoes_maximas = pyo.Constraint(model.P, rule=restricao_maxima)


In [180]:
# ------------------------------
# Escrita do Modelo em Arquivo
# ------------------------------
model.write("ex6.lp", io_options={"symbolic_solver_labels": True})

('ex6.lp', 2676203538992)

In [181]:
# ------------------------------
# Resolução
# ------------------------------
solver = pyo.SolverFactory("appsi_highs")
results = solver.solve(model, tee=True)

# Exibindo resultados
print("\nStatus do solver:", results.solver.status)
print("Condição de terminação:", results.solver.termination_condition)
valor_obj = pyo.value(model.objetivo)
formatted = f"{valor_obj:,.2f}"
formatted = formatted.replace(",", "X").replace(".", ",").replace("X", ".")
print("Valor da Função Objetivo: R$", formatted)



LP   has 20 rows; 5 cols; 60 nonzeros
Coefficient ranges:
  Matrix [1e-01, 1e+00]
  Cost   [3e+01, 5e+01]
  Bound  [0e+00, 0e+00]
  RHS    [4e+01, 5e+03]
Presolving model
9 rows, 5 cols, 45 nonzeros  0s
9 rows, 5 cols, 45 nonzeros  0s
Presolve : Reductions: rows 9(-11); columns 5(-0); elements 45(-15)
Solving the presolved LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0     0.0000000000e+00 Ph1: 0(0) 0s
          4    -3.2475000000e+04 Pr: 0(0) 0s
Solving the original LP from the solution after postsolve
Model status        : Optimal
Simplex   iterations: 4
Objective value     :  3.2475000000e+04
Relative P-D gap    :  2.2404796349e-16
HiGHS run time      :          0.00

Status do solver: ok
Condição de terminação: optimal
Valor da Função Objetivo: R$ 32.475,00


In [182]:
for i in model.P:    
    val = pyo.value(model.x[i])
    if val is not None and val > 1e-6:
        print(f"  Quantidade do produto {i} : {val:.2f}")

  Quantidade do produto Geladeira : 475.00
  Quantidade do produto Freezer : 50.00
  Quantidade do produto Fogao : 50.00
  Quantidade do produto LavaLoucas : 50.00
  Quantidade do produto Microondas : 75.00


## EX7

In [183]:
# Criação do modelo
model = pyo.ConcreteModel()

In [184]:

# Conjuntos
model.P = pyo.Set(initialize=['Super', 'Extra', 'Comum'])  # Modelos de carros
model.S = pyo.Set(initialize=['1', '2', '3'])  # operaçãoes

In [185]:
# ------------------------------
# Parâmetros
# ------------------------------
lucro = {'Super':5, 'Extra':4, 'Comum':3}
model.lucro = pyo.Param(model.P,initialize=lucro)


tempo_operacao = {
    ('Super', '1'): 6, ('Super', '2'): 3, ('Super', '3'): 4,
    ('Extra', '1'): 4, ('Extra', '2'): 6, ('Extra', '3'): 10,
    ('Comum', '1'): 2, ('Comum', '2'): 3, ('Comum', '3'): 4
} 
model.tempo_operacao = pyo.Param(model.P,model.S, initialize=tempo_operacao)

tempo_disponiveis = {'1': 240, '2': 180, '3': 200}
model.tempo_disponiveis = pyo.Param(model.S, initialize=tempo_disponiveis)



In [186]:

# ------------------------------
# Variáveis de Decisão
# ------------------------------
model.x = pyo.Var(model.P, domain=pyo.NonNegativeReals)  # Quantidade produzida de cada modelo


In [187]:

# ------------------------------
# Função Objetivo
# ------------------------------
def lucro_total(model):
    return sum(model.lucro[j] * model.x[j] for j in model.P)
model.objetivo = pyo.Objective(rule=lucro_total, sense=pyo.maximize)


In [188]:
# ------------------------------
# Restrições
# ------------------------------


# 1. Restrições de capacidade por operação
def restricao_capacidade(model, i):
    return sum(model.tempo_operacao[j,i] * model.x[j] for j in model.P) <= model.tempo_disponiveis[i]
model.capacidade = pyo.Constraint(model.S, rule=restricao_capacidade)


In [189]:
# ------------------------------
# Escrita do Modelo em Arquivo
# ------------------------------
model.write("ex7.lp", io_options={"symbolic_solver_labels": True})

('ex7.lp', 2676203526944)

In [190]:
# ------------------------------
# Resolução
# ------------------------------
solver = pyo.SolverFactory("appsi_highs")
results = solver.solve(model, tee=True)

# Exibindo resultados
print("\nStatus do solver:", results.solver.status)
print("Condição de terminação:", results.solver.termination_condition)
valor_obj = pyo.value(model.objetivo)
formatted = f"{valor_obj:,.2f}"
formatted = formatted.replace(",", "X").replace(".", ",").replace("X", ".")
print("Valor da Função Objetivo: R$", formatted)



LP   has 3 rows; 3 cols; 9 nonzeros
Coefficient ranges:
  Matrix [2e+00, 1e+01]
  Cost   [3e+00, 5e+00]
  Bound  [0e+00, 0e+00]
  RHS    [2e+02, 2e+02]
Presolving model
3 rows, 3 cols, 9 nonzeros  0s
3 rows, 3 cols, 9 nonzeros  0s
Presolve : Reductions: rows 3(-0); columns 3(-0); elements 9(-0) - Not reduced
Problem not reduced by presolve: solving the LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0    -3.2499955273e+00 Ph1: 3(10.25); Du: 3(3.25) 0s
          2     2.2000000000e+02 Pr: 0(0) 0s
Model status        : Optimal
Simplex   iterations: 2
Objective value     :  2.2000000000e+02
Relative P-D gap    :  0.0000000000e+00
HiGHS run time      :          0.00

Status do solver: ok
Condição de terminação: optimal
Valor da Função Objetivo: R$ 220,00


In [191]:
for i in model.P:    
    val = pyo.value(model.x[i])
    if val is not None and val > 1e-6:
        print(f"  Quantidade do produto {i} : {val:.2f}")

  Quantidade do produto Super : 35.00
  Quantidade do produto Comum : 15.00


## Ex8

In [192]:
model = pyo.ConcreteModel()

In [193]:

# Conjuntos
model.P = pyo.Set(initialize=[1, 2, 3])  
model.S = pyo.Set(initialize=['Montagem', 'Acabamento'])  

In [194]:
# ------------------------------
# Parâmetros
# ------------------------------
C = {1: 50, 2: 90, 3: 120}
model.C = pyo.Param(model.P,initialize=C)

CT = {1: 65, 2: 92, 3: 140}  # Custo unitário produção terceirizada
model.CT = pyo.Param(model.P,initialize=CT)

D = {1: 3000, 2: 2500, 3: 500}  # Demanda de cada motor
model.D = pyo.Param(model.P,initialize=D)

tempo_operacao = {(1, 'Montagem'): 1, (1, 'Acabamento'): 2.5,
     (2, 'Montagem'): 2, (2, 'Acabamento'): 1,
     (3, 'Montagem'): 0.5, (3, 'Acabamento'): 4}
model.tempo_operacao = pyo.Param(model.P,model.S, initialize=tempo_operacao)

tempo_disponiveis = {'Montagem': 6000, 'Acabamento': 10000}
model.tempo_disponiveis = pyo.Param(model.S, initialize=tempo_disponiveis)



In [195]:

# ------------------------------
# Variáveis de Decisão
# ------------------------------
model.x = pyo.Var(model.P, domain=pyo.NonNegativeReals) 
model.y = pyo.Var(model.P, domain=pyo.NonNegativeReals) 

In [196]:

# ------------------------------
# Função Objetivo
# ------------------------------
def fo(model):
    return sum(model.C[j] * model.x[j] + model.CT[j] * model.y[j] for j in model.P)
model.objetivo = pyo.Objective(rule=fo, sense=pyo.minimize)


In [197]:
# ------------------------------
# Restrições
# ------------------------------


# 1. Restrições de capacidade por operação
def restricao_capacidade(model, i):
    return sum(model.tempo_operacao[j,i] * model.x[j] for j in model.P) <= model.tempo_disponiveis[i]
model.capacidade = pyo.Constraint(model.S, rule=restricao_capacidade)

# 2. Demanda
def restricao_demanda(model, i):
    return model.x[i]+model.y[i] == model.D[i]
model.capacidade = pyo.Constraint(model.P, rule=restricao_demanda)


'pyomo.core.base.constraint.IndexedConstraint'>) on block unknown with a new
Component (type=<class 'pyomo.core.base.constraint.IndexedConstraint'>). This
is usually indicative of a modelling error. To avoid this warning, use
block.del_component() and block.add_component().


In [198]:
# ------------------------------
# Escrita do Modelo em Arquivo
# ------------------------------
model.write("ex8.lp", io_options={"symbolic_solver_labels": True})

('ex8.lp', 2676203676288)

In [199]:
# ------------------------------
# Resolução
# ------------------------------
solver = pyo.SolverFactory("appsi_highs")
results = solver.solve(model, tee=True)

# Exibindo resultados
print("\nStatus do solver:", results.solver.status)
print("Condição de terminação:", results.solver.termination_condition)
valor_obj = pyo.value(model.objetivo)
formatted = f"{valor_obj:,.2f}"
formatted = formatted.replace(",", "X").replace(".", ",").replace("X", ".")
print("Valor da Função Objetivo: R$", formatted)

LP   has 3 rows; 6 cols; 6 nonzeros
Coefficient ranges:
  Matrix [1e+00, 1e+00]
  Cost   [5e+01, 1e+02]
  Bound  [0e+00, 0e+00]
  RHS    [5e+02, 3e+03]
Presolving model
0 rows, 0 cols, 0 nonzeros  0s
0 rows, 0 cols, 0 nonzeros  0s
Presolve : Reductions: rows 0(-3); columns 0(-6); elements 0(-6) - Reduced to empty
Solving the original LP from the solution after postsolve
Model status        : Optimal
Objective value     :  4.3500000000e+05
Relative P-D gap    :  0.0000000000e+00
HiGHS run time      :          0.00

Status do solver: ok
Condição de terminação: optimal
Valor da Função Objetivo: R$ 435.000,00


In [200]:
for i in model.P:    
    val = pyo.value(model.x[i])
    val2 = pyo.value(model.y[i])
    
    print(f" \n Quantidade do produto {i} produção própria : {val:.2f}")
    print(f"  Quantidade do produto {i} produção terceirizada : {val2:.2f}")
    print(f"  Demanda atendida do produto {i} : {val + val2:.2f}")


 
 Quantidade do produto 1 produção própria : 3000.00
  Quantidade do produto 1 produção terceirizada : 0.00
  Demanda atendida do produto 1 : 3000.00
 
 Quantidade do produto 2 produção própria : 2500.00
  Quantidade do produto 2 produção terceirizada : 0.00
  Demanda atendida do produto 2 : 2500.00
 
 Quantidade do produto 3 produção própria : 500.00
  Quantidade do produto 3 produção terceirizada : 0.00
  Demanda atendida do produto 3 : 500.00


## Ex9

In [201]:
# Criação do modelo
model = pyo.ConcreteModel()

In [202]:

# Conjuntos
model.P = pyo.Set(initialize=['A', 'B', 'C'])  
model.S = pyo.Set(initialize=['Torno', 'Fresa', 'Furadeira'])  

In [203]:
# ------------------------------
# Parâmetros
# ------------------------------
C = {'A': 20, 'B': 15, 'C': 18}
model.C = pyo.Param(model.P,initialize=C)

D = {'A': 40, 'B': 50, 'C': 20}  # Demanda de cada motor
model.D = pyo.Param(model.P,initialize=D)

tempo_operacao = {
    ('A', 'Torno'): 5, ('A', 'Fresa'): 8, ('A', 'Furadeira'): 2,
    ('B', 'Torno'): 3, ('B', 'Fresa'): 4, ('B', 'Furadeira'): 5,
    ('C', 'Torno'): 5, ('C', 'Fresa'): 0, ('C', 'Furadeira'): 3
}
model.tempo_operacao = pyo.Param(model.P,model.S, initialize=tempo_operacao)

tempo_disponiveis = {'Torno': 400, 'Fresa': 500, 'Furadeira': 300}
model.tempo_disponiveis = pyo.Param(model.S, initialize=tempo_disponiveis)



In [204]:

# ------------------------------
# Variáveis de Decisão
# ------------------------------
model.x = pyo.Var(model.P, domain=pyo.NonNegativeReals) 

In [205]:

# ------------------------------
# Função Objetivo
# ------------------------------
def fo(model):
    return sum(model.C[j] * model.x[j] for j in model.P)
model.objetivo = pyo.Objective(rule=fo, sense=pyo.maximize)


In [206]:
# ------------------------------
# Restrições
# ------------------------------


# 1. Restrições de capacidade por operação
def restricao_capacidade(model, i):
    return sum(model.tempo_operacao[j,i] * model.x[j] for j in model.P) <= model.tempo_disponiveis[i]
model.capacidade = pyo.Constraint(model.S, rule=restricao_capacidade)

# 2. Demanda
def restricao_demanda(model, i):
    return model.x[i] <= model.D[i]
model.restricao_demanda = pyo.Constraint(model.P, rule=restricao_demanda)


In [207]:
# ------------------------------
# Escrita do Modelo em Arquivo
# ------------------------------
model.write("ex9.lp", io_options={"symbolic_solver_labels": True})

('ex9.lp', 2676203637056)

In [208]:
# ------------------------------
# Resolução
# ------------------------------
solver = pyo.SolverFactory("appsi_highs")
results = solver.solve(model, tee=True)

# Exibindo resultados
print("\nStatus do solver:", results.solver.status)
print("Condição de terminação:", results.solver.termination_condition)
valor_obj = pyo.value(model.objetivo)
formatted = f"{valor_obj:,.2f}"
formatted = formatted.replace(",", "X").replace(".", ",").replace("X", ".")
print("Valor da Função Objetivo: R$", formatted)

LP   has 6 rows; 3 cols; 11 nonzeros
Coefficient ranges:
  Matrix [1e+00, 8e+00]
  Cost   [2e+01, 2e+01]
  Bound  [0e+00, 0e+00]
  RHS    [2e+01, 5e+02]
Presolving model
3 rows, 3 cols, 8 nonzeros  0s
3 rows, 3 cols, 8 nonzeros  0s
Presolve : Reductions: rows 3(-3); columns 3(-0); elements 8(-3)
Solving the presolved LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0     0.0000000000e+00 Ph1: 0(0) 0s
          1    -1.6400000000e+03 Pr: 0(0) 0s
Solving the original LP from the solution after postsolve
Model status        : Optimal
Simplex   iterations: 1
Objective value     :  1.6400000000e+03
Relative P-D gap    :  0.0000000000e+00
HiGHS run time      :          0.00

Status do solver: ok
Condição de terminação: optimal
Valor da Função Objetivo: R$ 1.640,00


In [209]:
for i in model.P:    
    val = pyo.value(model.x[i])
    
    print(f"Quantidade do produto {i} produção própria : {val:.2f}")


Quantidade do produto A produção própria : 40.00
Quantidade do produto B produção própria : 32.00
Quantidade do produto C produção própria : 20.00


## Ex10- a)

In [210]:
# Criação do modelo
model = pyo.ConcreteModel()

In [211]:

# Conjuntos
model.meses = pyo.Set(initialize=[1, 2, 3, 4, 5, 6])  
 

In [212]:
# ------------------------------
# Parâmetros
# ------------------------------
demanda = {1: 5000, 2: 6000, 3: 5000, 4: 9000, 5: 7000, 6: 5000}  # Demanda de pares de sapatos por mês
model.demanda = pyo.Param(model.meses, initialize=demanda)
estoque_inicial = 1000
model.estoque_final_minimo = pyo.Param(initialize=estoque_inicial) # Estoque final mínimo desejado

tempo_producao_por_sapato = 0.5  # Horas por par de sapato
model.tempo_producao_por_sapato = pyo.Param(initialize=tempo_producao_por_sapato)

horas_trabalho_mensal = 150  # Horas disponíveis por operário
model.horas_trabalho_mensal = pyo.Param(initialize=horas_trabalho_mensal)

horas_extras_mensais = 30  # Máximo de horas extras permitidas por operário
model.horas_extras_mensais = pyo.Param(initialize=horas_extras_mensais)

max_operarios = 20  # Número máximo de operários simultaneamente empregados
model.max_operarios = pyo.Param(initialize=max_operarios)

custo_operario = 2000  # Custo fixo mensal por operário
model.custo_operario = pyo.Param(initialize=custo_operario)

custo_hora_extra = 50  # Custo por hora extra trabalhada
model.custo_hora_extra = pyo.Param(initialize=custo_hora_extra)

custo_estoque = 1  # Custo por sapato armazenado ao final do mês
model.custo_estoque = pyo.Param(initialize=custo_estoque)

In [213]:

# ------------------------------
# Variáveis de Decisão
# ------------------------------
model.O =  pyo.Var(model.meses, domain= pyo.NonNegativeIntegers)  # Número de operários empregados a cada mês
model.HE =  pyo.Var(model.meses, domain= pyo.NonNegativeReals)  # Total de horas extras trabalhadas a cada mês
model.E =  pyo.Var(model.meses, domain= pyo.NonNegativeReals)  # Nível de estoque ao fim de cada mês


In [214]:

# ------------------------------
# Função Objetivo
# ------------------------------
def fo(model):
    return sum(model.custo_operario * model.O[t] + model.custo_hora_extra * model.HE[t] + model.custo_estoque * model.E[t] for t in model.meses)
model.objetivo = pyo.Objective(rule=fo, sense=pyo.minimize)


In [215]:
# ------------------------------
# Restrições
# ------------------------------


# 1. Restrições de estoque
def restricao_estoque(model, t):
    if t == 1:
        return model.E[t] == estoque_inicial + (model.O[t] * horas_trabalho_mensal + model.HE[t]) / tempo_producao_por_sapato - demanda[t]
    else:
        return model.E[t] == model.E[t-1] + (model.O[t] * model.horas_trabalho_mensal + model.HE[t]) / model.tempo_producao_por_sapato - model.demanda[t]
model.restricoes_estoque = pyo.Constraint(model.meses, rule=restricao_estoque)

# 2. Restrição do número máximo de operários
def restricao_operarios(model, t):
    return model.O[t] <= model.max_operarios
model.restricoes_operarios = pyo.Constraint(model.meses, rule=restricao_operarios)

# 3. Restrição do número máximo de horas extras por operário
def restricao_horas_extras(model, t):
    return model.HE[t] <= model.O[t] * model.horas_extras_mensais
model.restricoes_horas_extras = pyo.Constraint(model.meses, rule=restricao_horas_extras)

# 4.  Restrição do estoque final mínimo
def restricao_estoque_final(model):
    return model.E[6] >= model.estoque_final_minimo
model.restricao_estoque_final = pyo.Constraint(rule=restricao_estoque_final)



In [216]:
# ------------------------------
# Escrita do Modelo em Arquivo
# ------------------------------
model.write("ex10.lp", io_options={"symbolic_solver_labels": True})

('ex10.lp', 2676203538656)

In [217]:
# ------------------------------
# Resolução
# ------------------------------
solver = pyo.SolverFactory("appsi_highs")
results = solver.solve(model, tee=True)

# Exibindo resultados
print("\nStatus do solver:", results.solver.status)
print("Condição de terminação:", results.solver.termination_condition)
valor_obj = pyo.value(model.objetivo)
formatted = f"{valor_obj:,.2f}"
formatted = formatted.replace(",", "X").replace(".", ",").replace("X", ".")


MIP  has 19 rows; 18 cols; 42 nonzeros; 6 integer variables (0 binary)
Coefficient ranges:
  Matrix [1e+00, 3e+02]
  Cost   [1e+00, 2e+03]
  Bound  [0e+00, 0e+00]
  RHS    [2e+01, 9e+03]
Presolving model
12 rows, 18 cols, 35 nonzeros  0s
12 rows, 18 cols, 35 nonzeros  0s

Solving MIP model with:
   12 rows
   18 cols (0 binary, 6 integer, 0 implied int., 12 continuous)
   35 nonzeros

Src: B => Branching; C => Central rounding; F => Feasibility pump; H => Heuristic; L => Sub-MIP;
     P => Empty MIP; R => Randomized rounding; S => Solve LP; T => Evaluate node; U => Unbounded;
     z => Trivial zero; l => Trivial lower; u => Trivial upper; p => Trivial point; X => User solution

        Nodes      |    B&B Tree     |            Objective Bounds              |  Dynamic Constraints |       Work      
Src  Proc. InQueue |  Leaves   Expl. | BestBound       BestSol              Gap |   Cuts   InLp Confl. | LpIters     Time

         0       0         0   0.00%   22800           inf          

In [218]:
print(f"Custo total mínimo: R$ {model.objetivo():.2f}")
for t in model.meses:
    print(f"Mês {t}: Operários = {model.O[t]():.0f}, Horas Extras = {model.HE[t]():.2f}, Estoque = {model.E[t]():.0f}")


Custo total mínimo: R$ 273000.00
Mês 1: Operários = 20, Horas Extras = 0.00, Estoque = 2000
Mês 2: Operários = 20, Horas Extras = 0.00, Estoque = 2000
Mês 3: Operários = 20, Horas Extras = 0.00, Estoque = 3000
Mês 4: Operários = 20, Horas Extras = 0.00, Estoque = 0
Mês 5: Operários = 20, Horas Extras = 500.00, Estoque = 0
Mês 6: Operários = 20, Horas Extras = 0.00, Estoque = 1000


## EX10- b
Para incorporar os custos de recrutamento e demissão no modelo anterior, devemos adicionar novas variáveis de decisão e modificar a função objetivo, o modelo matemático ficaria da seguinte forma

In [219]:
# Criação do modelo
model = pyo.ConcreteModel()

In [220]:

# Conjuntos
model.meses = pyo.Set(initialize=[1, 2, 3, 4, 5, 6])  


In [221]:
# ------------------------------
# Parâmetros
# ------------------------------
demanda = {1: 5000, 2: 6000, 3: 5000, 4: 9000, 5: 7000, 6: 5000}  # Demanda de pares de sapatos por mês
model.demanda = pyo.Param(model.meses, initialize=demanda)
estoque_inicial = 1000
model.estoque_final_minimo = pyo.Param(initialize=estoque_inicial) # Estoque final mínimo desejado

tempo_producao_por_sapato = 0.5  # Horas por par de sapato
model.tempo_producao_por_sapato = pyo.Param(initialize=tempo_producao_por_sapato)

horas_trabalho_mensal = 150  # Horas disponíveis por operário
model.horas_trabalho_mensal = pyo.Param(initialize=horas_trabalho_mensal)

horas_extras_mensais = 30  # Máximo de horas extras permitidas por operário
model.horas_extras_mensais = pyo.Param(initialize=horas_extras_mensais)

max_operarios = 20  # Número máximo de operários simultaneamente empregados
model.max_operarios = pyo.Param(initialize=max_operarios)

custo_operario = 2000  # Custo fixo mensal por operário
model.custo_operario = pyo.Param(initialize=custo_operario)

custo_hora_extra = 50  # Custo por hora extra trabalhada
model.custo_hora_extra = pyo.Param(initialize=custo_hora_extra)

custo_estoque = 1  # Custo por sapato armazenado ao final do mês
model.custo_estoque = pyo.Param(initialize=custo_estoque)

custo_recrutamento = 770
model.custo_recrutamento = pyo.Param(initialize=custo_recrutamento)
custo_demissao = 1000
model.custo_demissao = pyo.Param(initialize=custo_demissao)
operarios_iniciais = 15
model.operarios_iniciais = pyo.Param(initialize=operarios_iniciais)

In [222]:

# ------------------------------
# Variáveis de Decisão
# ------------------------------
model.O =  pyo.Var(model.meses, domain= pyo.NonNegativeIntegers)  # Número de operários empregados a cada mês
model.HE =  pyo.Var(model.meses, domain= pyo.NonNegativeReals)  # Total de horas extras trabalhadas a cada mês
model.E =  pyo.Var(model.meses, domain= pyo.NonNegativeReals)  # Nível de estoque ao fim de cada mês
model.R = pyo.Var(model.meses, domain=pyo.NonNegativeIntegers)  # Recrutamento
model.D = pyo.Var(model.meses, domain=pyo.NonNegativeIntegers)  # Demissões

In [223]:

# ------------------------------
# Função Objetivo
# ------------------------------
def fo(model):
    return sum(model.custo_operario * model.O[t] + model.custo_hora_extra * model.HE[t] + model.custo_estoque * model.E[t] +
               model.custo_recrutamento * model.R[t] + model.custo_demissao * model.D[t] for t in model.meses)
model.objetivo = pyo.Objective(rule=fo, sense=pyo.minimize)


In [224]:
# ------------------------------
# Restrições
# ------------------------------


# 1. Restrições de estoque
def restricao_estoque(model, t):
    if t == 1:
        return model.E[t] == estoque_inicial + (model.O[t] * horas_trabalho_mensal + model.HE[t]) / tempo_producao_por_sapato - demanda[t]
    else:
        return model.E[t] == model.E[t-1] + (model.O[t] * model.horas_trabalho_mensal + model.HE[t]) / model.tempo_producao_por_sapato - model.demanda[t]
model.restricoes_estoque = pyo.Constraint(model.meses, rule=restricao_estoque)

# 2. Restrição do número máximo de operários
def restricao_operarios_max(model, t):
    return model.O[t] <= model.max_operarios
model.restricao_operarios_max = pyo.Constraint(model.meses, rule=restricao_operarios_max)

# 3. Restrição do número máximo de horas extras por operário
def restricao_horas_extras(model, t):
    return model.HE[t] <= model.O[t] * model.horas_extras_mensais
model.restricoes_horas_extras = pyo.Constraint(model.meses, rule=restricao_horas_extras)

# 4.  Restrição do estoque final mínimo
def restricao_estoque_final(model):
    return model.E[6] >= model.estoque_final_minimo
model.restricao_estoque_final = pyo.Constraint(rule=restricao_estoque_final)

# 5. Restrição de número de operários
def restricao_operarios(model, t):
    if t == 1:
        return model.O[t] == operarios_iniciais + model.R[t] - model.D[t]
    else:
        return model.O[t] == model.O[t-1] + model.R[t] - model.D[t]
model.restricoes_operarios = pyo.Constraint(model.meses, rule=restricao_operarios)




In [225]:
# ------------------------------
# Escrita do Modelo em Arquivo
# ------------------------------
model.write("ex10b.lp", io_options={"symbolic_solver_labels": True})


('ex10b.lp', 2676203844368)

In [226]:
# ------------------------------
# Resolução
# ------------------------------
solver = pyo.SolverFactory("appsi_highs")
results = solver.solve(model, tee=True)

# Exibindo resultados
print("\nStatus do solver:", results.solver.status)
print("Condição de terminação:", results.solver.termination_condition)

MIP  has 25 rows; 30 cols; 65 nonzeros; 18 integer variables (0 binary)
Coefficient ranges:
  Matrix [1e+00, 3e+02]
  Cost   [1e+00, 2e+03]
  Bound  [0e+00, 0e+00]
  RHS    [2e+01, 9e+03]
Presolving model
18 rows, 30 cols, 58 nonzeros  0s
18 rows, 30 cols, 58 nonzeros  0s

Solving MIP model with:
   18 rows
   30 cols (0 binary, 18 integer, 0 implied int., 12 continuous)
   58 nonzeros

Src: B => Branching; C => Central rounding; F => Feasibility pump; H => Heuristic; L => Sub-MIP;
     P => Empty MIP; R => Randomized rounding; S => Solve LP; T => Evaluate node; U => Unbounded;
     z => Trivial zero; l => Trivial lower; u => Trivial upper; p => Trivial point; X => User solution

        Nodes      |    B&B Tree     |            Objective Bounds              |  Dynamic Constraints |       Work      
Src  Proc. InQueue |  Leaves   Expl. | BestBound       BestSol              Gap |   Cuts   InLp Confl. | LpIters     Time

         0       0         0   0.00%   22800           inf        

In [227]:
print(f"Custo total mínimo: R$ {model.objetivo():.2f}")
for t in model.meses:
    print(f'\n Mês {t}:')
    print(f"Operários = {model.O[t]():.0f}, Horas Extras = {model.HE[t]():.2f}, Estoque = {model.E[t]():.0f}")
    print(f"Recrutamento = {model.R[t]():.0f}, Demissões = {model.D[t]():.0f}")

Custo total mínimo: R$ 276850.00

 Mês 1:
Operários = 20, Horas Extras = 0.00, Estoque = 2000
Recrutamento = 5, Demissões = 0

 Mês 2:
Operários = 20, Horas Extras = 0.00, Estoque = 2000
Recrutamento = -0, Demissões = 0

 Mês 3:
Operários = 20, Horas Extras = 0.00, Estoque = 3000
Recrutamento = 0, Demissões = 0

 Mês 4:
Operários = 20, Horas Extras = 0.00, Estoque = 0
Recrutamento = 0, Demissões = 0

 Mês 5:
Operários = 20, Horas Extras = 500.00, Estoque = 0
Recrutamento = 0, Demissões = 0

 Mês 6:
Operários = 20, Horas Extras = 0.00, Estoque = 1000
Recrutamento = 0, Demissões = 0


## Ex11- a)

In [228]:
# Criação do modelo
model = pyo.ConcreteModel()

In [229]:

# Conjuntos
model.S = pyo.Set(initialize=[1, 2, 3, 4])
 

In [230]:
# ------------------------------
# Parâmetros
# ------------------------------
# Custos de produção por unidade
custo_producao = {1: 2, 2: 2, 3: 2.5, 4: 2.5}
model.c_prod = pyo.Param(model.S, initialize=custo_producao)

# Custos de produção em horas extras (apenas semanas 1 e 2)
custo_extra = {1: 2.8, 2: 2.8}
model.c_extra = pyo.Param(model.S, initialize=custo_extra, default=0)

# Custo de armazenamento por unidade por semana
custo_estoque = 0.2

# Demanda semanal de camisetas
demanda = {1: 5000, 2: 10000, 3: 30000, 4: 60000}
model.d = pyo.Param(model.S, initialize=demanda)

# Capacidade máxima de produção regular
capacidade_producao = 25000

# Capacidade extra de produção nas semanas 1 e 2
capacidade_extra = 10000



In [231]:
# ------------------------------
# Variáveis de Decisão
# ------------------------------
# Quantidade produzida regularmente por semana
model.x = pyo.Var(model.S, domain=pyo.NonNegativeIntegers)

# Quantidade produzida com horas extras (apenas semanas 1 e 2)
model.y = pyo.Var(model.S, domain=pyo.NonNegativeIntegers)

# Quantidade estocada ao final de cada semana
model.s = pyo.Var(model.S, domain=pyo.NonNegativeIntegers)

In [232]:

# ------------------------------
# Função Objetivo
# ------------------------------
def fo(model):
    custo_total_producao = sum(model.x[s] * model.c_prod[s] + model.y[s] * model.c_extra[s] for s in model.S)
    custo_total_estoque = sum(model.s[s] * custo_estoque for s in model.S if s < 4)
    return custo_total_producao + custo_total_estoque
model.objetivo = pyo.Objective(rule=fo, sense=pyo.minimize)


In [233]:
# ------------------------------
# Restrições
# ------------------------------

# 1. Limite de Produção Regular
def limite_producao(model, s):
    return model.x[s] <= capacidade_producao

model.CapProd = pyo.Constraint(model.S, rule=limite_producao)

# 2. Limite de Produção Extra nas semanas 1 e 2
def limite_extra(model, s):
    if s in [1, 2]:
        return model.y[s] <= capacidade_extra
    return model.y[s] == 0  # Sem produção extra nas semanas 3 e 4

model.CapExtra = pyo.Constraint(model.S, rule=limite_extra)

# 3. Balanço de Estoque
def estoque(model, s):
    if s == 1:
        return model.x[1] + model.y[1] == model.d[1] + model.s[1]
    elif s == 2:
        return model.x[2] + model.y[2] + model.s[1] == model.d[2] + model.s[2]
    elif s == 3:
        return model.x[3] + model.s[2] == model.d[3] + model.s[3]
    elif s == 4:
        return model.x[4] + model.s[3] == model.d[4]

model.BalancoEstoque = pyo.Constraint(model.S, rule=estoque)



In [234]:
# ------------------------------
# Escrita do Modelo em Arquivo
# ------------------------------
model.write("ex11a.lp", io_options={"symbolic_solver_labels": True})

('ex11a.lp', 2676203693920)

In [235]:
# ------------------------------
# Resolução
# ------------------------------
solver = pyo.SolverFactory("appsi_highs")
results = solver.solve(model, tee=True)

# Exibindo resultados
print("\nStatus do solver:", results.solver.status)
print("Condição de terminação:", results.solver.termination_condition)
valor_obj = pyo.value(model.objetivo)
formatted = f"{valor_obj:,.2f}"
formatted = formatted.replace(",", "X").replace(".", ",").replace("X", ".")
print("Valor da Função Objetivo: R$", formatted)

MIP  has 12 rows; 11 cols; 20 nonzeros; 11 integer variables (0 binary)
Coefficient ranges:
  Matrix [1e+00, 1e+00]
  Cost   [2e-01, 3e+00]
  Bound  [0e+00, 0e+00]
  RHS    [5e+03, 6e+04]
Presolving model
3 rows, 8 cols, 10 nonzeros  0s
2 rows, 7 cols, 8 nonzeros  0s
1 rows, 6 cols, 6 nonzeros  0s
1 rows, 4 cols, 4 nonzeros  0s
1 rows, 3 cols, 3 nonzeros  0s
1 rows, 3 cols, 3 nonzeros  0s
Objective function is integral with scale 10

Solving MIP model with:
   1 rows
   3 cols (0 binary, 3 integer, 0 implied int., 0 continuous)
   3 nonzeros

Src: B => Branching; C => Central rounding; F => Feasibility pump; H => Heuristic; L => Sub-MIP;
     P => Empty MIP; R => Randomized rounding; S => Solve LP; T => Evaluate node; U => Unbounded;
     z => Trivial zero; l => Trivial lower; u => Trivial upper; p => Trivial point; X => User solution

        Nodes      |    B&B Tree     |            Objective Bounds              |  Dynamic Constraints |       Work      
Src  Proc. InQueue |  Leaves  

In [236]:
print(f"Custo total mínimo: R$ {model.objetivo():.2f}")
for t in model.S:
    print(f"\nSemana {t}:")
    print(f"Produção regular na semana {t}: {model.x[t].value} unidades")
    print(f"Produção extra na semana {t}: {model.y[t].value} unidades")
    print(f"Estoque ao final da semana {t}: {model.s[t].value} unidades")
    


Custo total mínimo: R$ 258000.00

Semana 1:
Produção regular na semana 1: 25000.0 unidades
Produção extra na semana 1: 0.0 unidades
Estoque ao final da semana 1: 20000.0 unidades

Semana 2:
Produção regular na semana 2: 25000.0 unidades
Produção extra na semana 2: 5000.0 unidades
Estoque ao final da semana 2: 40000.0 unidades

Semana 3:
Produção regular na semana 3: 25000.0 unidades
Produção extra na semana 3: 0.0 unidades
Estoque ao final da semana 3: 35000.0 unidades

Semana 4:
Produção regular na semana 4: 25000.0 unidades
Produção extra na semana 4: 0.0 unidades
Estoque ao final da semana 4: None unidades


## Ex_ 11b

In [237]:
# Criação do modelo
model = pyo.ConcreteModel()

In [238]:

# Conjuntos
model.S = pyo.Set(initialize=[1, 2, 3, 4])  # Semanas
model.T = pyo.Set(initialize=["A", "B", "C", "D"])  # Times


In [239]:
# ------------------------------
# Parâmetros
# ------------------------------
# Preço de venda das camisetas por semana
preco_venda = {
    (1, "A"): 5, (2, "A"): 6, (3, "A"): 3, (4, "A"): 0,
    (1, "B"): 5, (2, "B"): 6, (3, "B"): 3, (4, "B"): 0,
    (1, "C"): 5, (2, "C"): 6, (3, "C"): 8, (4, "C"): 9,
    (1, "D"): 5, (2, "D"): 6, (3, "D"): 8, (4, "D"): 9
}
model.v = pyo.Param(model.S, model.T, initialize=preco_venda)

# Demanda por semana e time
demanda = {
    (1, "A"): 1250, (2, "A"): 2500, (3, "A"): 5000, (4, "A"): 0,
    (1, "B"): 1250, (2, "B"): 2500, (3, "B"): 5000, (4, "B"): 0,
    (1, "C"): 1250, (2, "C"): 2500, (3, "C"): 14500, (4, "C"): 30000,
    (1, "D"): 1250, (2, "D"): 2500, (3, "D"): 14500, (4, "D"): 30000
}
model.d = pyo.Param(model.S, model.T, initialize=demanda)

# Custos de produção
custo_producao = {1: 2, 2: 2, 3: 2.5, 4: 2.5}
model.c_prod = pyo.Param(model.S, initialize=custo_producao)

# Custos de produção em horas extras (apenas semanas 1 e 2)
custo_extra = {1: 2.8, 2: 2.8}
model.c_extra = pyo.Param(model.S, initialize=custo_extra, default=0)

# Custo de armazenamento por unidade por semana
custo_estoque = 0.2

# Capacidade máxima de produção regular
capacidade_producao = 25000

# Produção extra disponível nas semanas 1 e 2
capacidade_extra = 10000



In [240]:

# ------------------------------
# Variáveis de Decisão
# ------------------------------
# Quantidade produzida por semana e time
model.x = pyo.Var(model.S, model.T, domain=pyo.NonNegativeIntegers)

# Quantidade estocada ao final de cada semana por time
model.s = pyo.Var(model.S, model.T, domain=pyo.NonNegativeIntegers)

# Quantidade produzida com horas extras (apenas semanas 1 e 2)
model.y = pyo.Var(model.S, domain=pyo.NonNegativeIntegers)

In [241]:

# ------------------------------
# Função Objetivo
# ------------------------------
def fo(model):
    
    receita_total = sum(model.v[s, t] * model.x[s, t] for s in model.S for t in model.T)

    
    custo_total_producao = sum(model.x[s, t] * model.c_prod[s] for s in model.S for t in model.T)
    custo_total_extra = sum(model.y[s] * model.c_extra[s] for s in model.S if s in [1, 2])
    custo_total_estoque = sum(model.s[s, t] * custo_estoque for s in model.S for t in model.T if s < 4)

    return receita_total - (custo_total_producao + custo_total_extra + custo_total_estoque)
model.objetivo = pyo.Objective(rule=fo, sense=pyo.maximize)


In [242]:
# ------------------------------
# Restrições
# ------------------------------
# 1. Limite de produção por semana
def limite_producao(model, s):
    return sum(model.x[s, t] for t in model.T) + (model.y[s] if s in [1, 2] else 0) <= (capacidade_producao + (capacidade_extra if s in [1, 2] else 0))

model.CapProd = pyo.Constraint(model.S, rule=limite_producao)

# 2. Balanço de estoque por time
def estoque(model, s, t):
    if s == 1:
        return model.x[1, t] == model.d[1, t] + model.s[1, t]
    elif s == 2:
        return model.x[2, t] + model.s[1, t] == model.d[2, t] + model.s[2, t]
    elif s == 3:
        return model.x[3, t] + model.s[2, t] == model.d[3, t] + model.s[3, t]
    elif s == 4:
        return model.x[4, t] + model.s[3, t] == model.d[4, t]

model.BalancoEstoque = pyo.Constraint(model.S, model.T, rule=estoque)

In [243]:
# ------------------------------
# Escrita do Modelo em Arquivo
# ------------------------------
model.write("ex11b.lp", io_options={"symbolic_solver_labels": True})


('ex11b.lp', 2676203505504)

In [244]:
# ------------------------------
# Resolução
# ------------------------------
solver = pyo.SolverFactory("appsi_highs")
results = solver.solve(model, tee=True)

# Exibindo resultados
print("\nStatus do solver:", results.solver.status)
print("Condição de terminação:", results.solver.termination_condition)
valor_obj = pyo.value(model.objetivo)
formatted = f"{valor_obj:,.2f}"
formatted = formatted.replace(",", "X").replace(".", ",").replace("X", ".")
print("Valor da Função Objetivo: R$", formatted)

MIP  has 20 rows; 30 cols; 58 nonzeros; 30 integer variables (0 binary)
Coefficient ranges:
  Matrix [1e+00, 1e+00]
  Cost   [2e-01, 6e+00]
  Bound  [0e+00, 0e+00]
  RHS    [1e+03, 4e+04]
Presolving model
12 rows, 18 cols, 36 nonzeros  0s
10 rows, 16 cols, 32 nonzeros  0s
Objective function is integral with scale 10

Solving MIP model with:
   10 rows
   16 cols (0 binary, 16 integer, 0 implied int., 0 continuous)
   32 nonzeros

Src: B => Branching; C => Central rounding; F => Feasibility pump; H => Heuristic; L => Sub-MIP;
     P => Empty MIP; R => Randomized rounding; S => Solve LP; T => Evaluate node; U => Unbounded;
     z => Trivial zero; l => Trivial lower; u => Trivial upper; p => Trivial point; X => User solution

        Nodes      |    B&B Tree     |            Objective Bounds              |  Dynamic Constraints |       Work      
Src  Proc. InQueue |  Leaves   Expl. | BestBound       BestSol              Gap |   Cuts   InLp Confl. | LpIters     Time

         0       0    

In [245]:
print("\nResultados por semana:")
for s in model.S:
    print(f"\n Semana {s}")
    
    print("Produção por time:")
    for t in model.T:
        print(f"  - Time {t}: {model.x[s, t].value} camisetas")
    print("Estoque final por time:")
    for t in model.T:
        print(f"  - Time {t}: {model.s[s, t].value} camisetas")
    print(f"Produção extra na semana {s}: {model.y[s].value} camisetas")



Resultados por semana:

 Semana 1
Produção por time:
  - Time A: 1250.0 camisetas
  - Time B: 8750.0 camisetas
  - Time C: 1250.0 camisetas
  - Time D: 17750.0 camisetas
Estoque final por time:
  - Time A: 0.0 camisetas
  - Time B: 7500.0 camisetas
  - Time C: 0.0 camisetas
  - Time D: 16500.0 camisetas
Produção extra na semana 1: 0.0 camisetas

 Semana 2
Produção por time:
  - Time A: 7500.0 camisetas
  - Time B: -0.0 camisetas
  - Time C: 27500.0 camisetas
  - Time D: 0.0 camisetas
Estoque final por time:
  - Time A: 5000.0 camisetas
  - Time B: 5000.0 camisetas
  - Time C: 25000.0 camisetas
  - Time D: 14000.0 camisetas
Produção extra na semana 2: 0.0 camisetas

 Semana 3
Produção por time:
  - Time A: 0.0 camisetas
  - Time B: 0.0 camisetas
  - Time C: 0.0 camisetas
  - Time D: 25000.0 camisetas
Estoque final por time:
  - Time A: 0.0 camisetas
  - Time B: 0.0 camisetas
  - Time C: 10500.0 camisetas
  - Time D: 24500.0 camisetas
Produção extra na semana 3: None camisetas

 Semana 

## Ex12

In [246]:
# Criação do modelo
model = pyo.ConcreteModel()

In [247]:

# Conjuntos
model.F = pyo.Set(initialize=["SP", "JP", "MA"])  
model.R = pyo.Set(initialize=["RJ", "Salvador", "Aracaju", "Maceió", "Recife"]) 


In [248]:
# Capacidade de produção nas fábricas
capacidade_producao = {"SP": 10000, "JP": 5000, "MA": 6000}
model.cap = pyo.Param(model.F, initialize=capacidade_producao)

# Demanda dos pontos de revenda
demanda_revenda = {"RJ": 6000, "Salvador": 5000, "Aracaju": 2000, "Maceió": 1000, "Recife": 3000}
model.demanda = pyo.Param(model.R, initialize=demanda_revenda)

# Custos de transporte (R$/unidade)
custo_transporte = {
    ("SP", "RJ"): 1.00, ("SP", "Salvador"): 2.00, ("SP", "Aracaju"): 3.00, ("SP", "Maceió"): 3.50, ("SP", "Recife"): 4.00,
    ("JP", "RJ"): 4.00, ("JP", "Salvador"): 2.00, ("JP", "Aracaju"): 1.50, ("JP", "Maceió"): 1.20, ("JP", "Recife"): 1.00,
    ("MA", "RJ"): 6.00, ("MA", "Salvador"): 4.00, ("MA", "Aracaju"): 3.50, ("MA", "Maceió"): 3.00, ("MA", "Recife"): 2.00
}
model.custo = pyo.Param(model.F, model.R, initialize=custo_transporte)

In [249]:

# ------------------------------
# Variáveis de Decisão
# ------------------------------

model.x = pyo.Var(model.F, model.R, domain=pyo.NonNegativeIntegers)

In [250]:

# ------------------------------
# Função Objetivo
# ------------------------------
def fo(model):
    return sum(model.x[i, j] * model.custo[i, j] for i in model.F for j in model.R)/1000
model.objetivo = pyo.Objective(rule=fo, sense=pyo.minimize)


In [251]:
# ------------------------------
# Restrições
# ------------------------------

# 1. Capacidade de produção nas fábricas
def capacidade(model, i):
    return sum(model.x[i, j] for j in model.R) <= model.cap[i]

model.Capacidade = pyo.Constraint(model.F, rule=capacidade)

# 2. Atendimento da demanda dos pontos de revenda
def atendimento_demanda(model, j):
    return sum(model.x[i, j] for i in model.F) == model.demanda[j]
model.AtendimentoDemanda = pyo.Constraint(model.R, rule=atendimento_demanda)



In [252]:
# ------------------------------
# Escrita do Modelo em Arquivo
# ------------------------------
model.write("ex12.lp", io_options={"symbolic_solver_labels": True})

('ex12.lp', 2676179834000)

In [253]:
# ------------------------------
# Resolução
# ------------------------------
solver = pyo.SolverFactory("appsi_highs")
results = solver.solve(model, tee=True)

# Exibindo resultados
print("\nStatus do solver:", results.solver.status)
print("Condição de terminação:", results.solver.termination_condition)


MIP  has 8 rows; 15 cols; 30 nonzeros; 15 integer variables (0 binary)
Coefficient ranges:
  Matrix [1e+00, 1e+00]
  Cost   [1e-03, 6e-03]
  Bound  [0e+00, 0e+00]
  RHS    [1e+03, 1e+04]
Presolving model
8 rows, 15 cols, 30 nonzeros  0s
8 rows, 15 cols, 30 nonzeros  0s
Objective function is integral with scale 50000

Solving MIP model with:
   8 rows
   15 cols (0 binary, 15 integer, 0 implied int., 0 continuous)
   30 nonzeros

Src: B => Branching; C => Central rounding; F => Feasibility pump; H => Heuristic; L => Sub-MIP;
     P => Empty MIP; R => Randomized rounding; S => Solve LP; T => Evaluate node; U => Unbounded;
     z => Trivial zero; l => Trivial lower; u => Trivial upper; p => Trivial point; X => User solution

        Nodes      |    B&B Tree     |            Objective Bounds              |  Dynamic Constraints |       Work      
Src  Proc. InQueue |  Leaves   Expl. | BestBound       BestSol              Gap |   Cuts   InLp Confl. | LpIters     Time

         0       0     

In [254]:
for i in model.F:
    for j in model.R:
        val = pyo.value(model.x[i, j])
        if val > 0:  
            print(f"Quantidade transportada da fábrica {i} para {j}: {val:.2f}")

# Exibindo o valor da função objetivo
print("\nValor da Função Objetivo: R$", formatted)

Quantidade transportada da fábrica SP para RJ: 6000.00
Quantidade transportada da fábrica SP para Salvador: 4000.00
Quantidade transportada da fábrica JP para Salvador: 1000.00
Quantidade transportada da fábrica JP para Aracaju: 2000.00
Quantidade transportada da fábrica JP para Maceió: 1000.00
Quantidade transportada da fábrica JP para Recife: 1000.00
Quantidade transportada da fábrica MA para Recife: 2000.00

Valor da Função Objetivo: R$ 505.400,00


## Ex 13

In [255]:
# Criação do modelo
model = pyo.ConcreteModel()

In [256]:

# Conjuntos
model.M = pyo.Set(initialize=["Mat1", "Mat2", "Mat3", "Mat4", "Mat5", "Mat6", "Mat7"])

model.E = pyo.Set(initialize=["Fe", "Cu", "Mn", "Mg", "Al", "Si"])

In [257]:
custo_material = {"Mat1": 0.03, "Mat2": 0.08, "Mat3": 0.17, "Mat4": 0.12, 
                  "Mat5": 0.15, "Mat6": 0.21, "Mat7": 0.38}
model.c_material = pyo.Param(model.M, initialize=custo_material)

# Composição química das matérias-primas
qtd_elementos = {
    ("Mat1", "Fe"): 0.15, ("Mat2", "Fe"): 0.04, ("Mat3", "Fe"): 0.02, ("Mat4", "Fe"): 0.04, ("Mat5", "Fe"): 0.02, ("Mat6", "Fe"): 0.01, ("Mat7", "Fe"): 0.03,
    ("Mat1", "Cu"): 0.03, ("Mat2", "Cu"): 0.05, ("Mat3", "Cu"): 0.08, ("Mat4", "Cu"): 0.02, ("Mat5", "Cu"): 0.06, ("Mat6", "Cu"): 0.01,
    ("Mat1", "Mn"): 0.02, ("Mat2", "Mn"): 0.04, ("Mat3", "Mn"): 0.01, ("Mat4", "Mn"): 0.02, ("Mat5", "Mn"): 0.02,
    ("Mat1", "Mg"): 0.02, ("Mat2", "Mg"): 0.03, ("Mat3", "Mg"): 0.00, ("Mat4", "Mg"): 0.00, ("Mat5", "Mg"): 0.01,
    ("Mat1", "Al"): 0.70, ("Mat2", "Al"): 0.75, ("Mat3", "Al"): 0.80, ("Mat4", "Al"): 0.75, ("Mat5", "Al"): 0.80, ("Mat6", "Al"): 0.97,
    ("Mat1", "Si"): 0.02, ("Mat2", "Si"): 0.06, ("Mat3", "Si"): 0.08, ("Mat4", "Si"): 0.12, ("Mat5", "Si"): 0.02, ("Mat6", "Si"): 0.01, ("Mat7", "Si"): 0.97,
}
model.q_material = pyo.Param(model.M, model.E, initialize=qtd_elementos, default=0)

# Limites de disponibilidade das matérias-primas
estoque_material = {"Mat1": 200, "Mat2": 750, "Mat3": 800, "Mat4": 700, "Mat5": 1500, "Mat6": float("inf"), "Mat7": float("inf")}
model.e_material = pyo.Param(model.M, initialize=estoque_material)

# Limites de presença de elementos químicos
limites_elementos = {"Fe": (0, 60), "Cu": (0, 100), "Mn": (0, 40), "Mg": (0, 30), "Al": (1500, float("inf")), "Si": (250, 300)}
model.l_elemento_min = pyo.Param(model.E, initialize={e: limites_elementos[e][0] for e in model.E})
model.l_elemento_max = pyo.Param(model.E, initialize={e: limites_elementos[e][1] for e in model.E})


In [258]:

# ------------------------------
# Variáveis de Decisão
# ------------------------------

model.x = pyo.Var(model.M, domain=pyo.NonNegativeReals)

In [259]:

# ------------------------------
# Função Objetivo
# ------------------------------
def fo(model):
    return sum(model.x[m] * model.c_material[m] for m in model.M)

model.objetivo = pyo.Objective(rule=fo, sense=pyo.minimize)


In [260]:
# ------------------------------
# Restrições
# ------------------------------

# 1. Limite total da liga
def liga_total_rule(model):
    return sum(model.x[m] for m in model.M) == 2000

model.liga_total = pyo.Constraint(rule=liga_total_rule)


# 2. Restrições de composição química
def elemento_min_rule(model, e):
    return sum(model.x[m] * model.q_material[m, e] for m in model.M) >= model.l_elemento_min[e]

def elemento_max_rule(model, e):
    return sum(model.x[m] * model.q_material[m, e] for m in model.M) <= model.l_elemento_max[e]

model.limite_elemento_min = pyo.Constraint(model.E, rule=elemento_min_rule)
model.limite_elemento_max = pyo.Constraint(model.E, rule=elemento_max_rule)

# 3. Limites de disponibilidade das matérias-primas
def estoque_rule(model, m):
    return model.x[m] <= model.e_material[m]

model.limite_estoque = pyo.Constraint(model.M, rule=estoque_rule)



In [261]:
# ------------------------------
# Escrita do Modelo em Arquivo
# ------------------------------
model.write("ex13.lp", io_options={"symbolic_solver_labels": True})

('ex13.lp', 2676203952256)

In [262]:
# ------------------------------
# Resolução
# ------------------------------
solver = pyo.SolverFactory("appsi_highs")
results = solver.solve(model, tee=True)

# Exibindo resultados
print("\nStatus do solver:", results.solver.status)
print("Condição de terminação:", results.solver.termination_condition)
valor_obj = pyo.value(model.objetivo)
formatted = f"{valor_obj:,.2f}"
formatted = formatted.replace(",", "X").replace(".", ",").replace("X", ".")
print("Valor da Função Objetivo: R$", formatted)

LP   has 20 rows; 7 cols; 82 nonzeros
Coefficient ranges:
  Matrix [1e-02, 1e+00]
  Cost   [3e-02, 4e-01]
  Bound  [0e+00, 0e+00]
  RHS    [3e+01, 2e+03]
Presolving model
8 rows, 7 cols, 48 nonzeros  0s
Dependent equations search running on 1 equations with time limit of 1000.00s
Dependent equations search removed 0 rows and 0 nonzeros in 0.00s (limit = 1000.00s)
7 rows, 7 cols, 41 nonzeros  0s
Presolve : Reductions: rows 7(-13); columns 7(-0); elements 41(-41)
Solving the presolved LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0     0.0000000000e+00 Pr: 3(687.5) 0s
          7     2.9621660650e+02 Pr: 0(0) 0s
Solving the original LP from the solution after postsolve
Model status        : Optimal
Simplex   iterations: 7
Objective value     :  2.9621660650e+02
Relative P-D gap    :  0.0000000000e+00
HiGHS run time      :          0.00

Status do solver: ok
Condição de terminação: optimal
Valor da Função Objetivo: R$ 296,22


In [263]:

for m in model.M:
    valor = pyo.value(model.x[m])
    print(f"Quantidade de {m}: {valor:.2f} kg")



Quantidade de Mat1: 0.00 kg
Quantidade de Mat2: 665.34 kg
Quantidade de Mat3: 490.25 kg
Quantidade de Mat4: 424.19 kg
Quantidade de Mat5: 0.00 kg
Quantidade de Mat6: 299.64 kg
Quantidade de Mat7: 120.58 kg


#Ex14

In [264]:
# Criação do modelo
model = pyo.ConcreteModel()

In [265]:

# Conjuntos

model.Empresas = pyo.Set(initialize=["D", "N"])

model.Produtos = pyo.Set(initialize=["Cerveja", "Refrigerante"])

In [266]:
# Bonificação esperada por empresa
bonificacao = {"D": 0.12, "N": 0.20}
model.bonificacao = pyo.Param(model.Empresas, initialize=bonificacao)

# Porcentagem de cada produto na empresa D
porcentagem_produto = {
    ("D", "Cerveja"): 0.40,
    ("D", "Refrigerante"): 0.60,
    ("N", "Cerveja"): 1.00
}
model.p_produto = pyo.Param(model.Empresas, model.Produtos, initialize=porcentagem_produto, default=0)

# Limite de investimento por empresa
limite_empresa = {"D": 270000, "N": 150000}
model.l_empresa = pyo.Param(model.Empresas, initialize=limite_empresa)

# Limite de investimento por produto
limite_produto = {"Cerveja": 180000, "Refrigerante": 180000}
model.l_produto = pyo.Param(model.Produtos, initialize=limite_produto)

# Orçamento total disponível
model.orcamento = pyo.Param(initialize=300000)


In [267]:

# ------------------------------
# Variáveis de Decisão
# ------------------------------

model.x = pyo.Var(model.Empresas, domain=pyo.NonNegativeReals)

In [268]:

# ------------------------------
# Função Objetivo
# ------------------------------
def fo(model):
    return sum(model.x[e] * model.bonificacao[e] for e in model.Empresas)

model.objetivo = pyo.Objective(rule=fo, sense=pyo.maximize)


In [269]:
# ------------------------------
# Restrições
# ------------------------------

# 1. Restrição de orçamento total
def orcamento_rule(model):
    return sum(model.x[e] for e in model.Empresas) <= model.orcamento
model.orcamento_total = pyo.Constraint(rule=orcamento_rule)

# 2. Restrição de limite de investimento por empresa
def limite_empresa_rule(model, e):
    return model.x[e] <= model.l_empresa[e]


model.limite_empresa = pyo.Constraint(model.Empresas, rule=limite_empresa_rule)

# 3. Restrição de limite de investimento por produto
def limite_produto_rule(model, p):
    return sum(model.x[e] * model.p_produto[e, p] for e in model.Empresas) <= model.l_produto[p]

model.limite_produto = pyo.Constraint(model.Produtos, rule=limite_produto_rule)




In [270]:
# ------------------------------
# Escrita do Modelo em Arquivo
# ------------------------------
model.write("ex14.lp", io_options={"symbolic_solver_labels": True})

('ex14.lp', 2676204097200)

In [271]:
# ------------------------------
# Resolução
# ------------------------------
solver = pyo.SolverFactory("appsi_highs")
results = solver.solve(model, tee=True)

# Exibindo resultados
print("\nStatus do solver:", results.solver.status)
print("Condição de terminação:", results.solver.termination_condition)
valor_obj = pyo.value(model.objetivo)
formatted = f"{valor_obj:,.2f}"
formatted = formatted.replace(",", "X").replace(".", ",").replace("X", ".")
print("Valor da Função Objetivo: R$", formatted)

LP   has 5 rows; 2 cols; 7 nonzeros
Coefficient ranges:
  Matrix [4e-01, 1e+00]
  Cost   [1e-01, 2e-01]
  Bound  [0e+00, 0e+00]
  RHS    [2e+05, 3e+05]
Presolving model
2 rows, 2 cols, 4 nonzeros  0s
2 rows, 2 cols, 4 nonzeros  0s
Presolve : Reductions: rows 2(-3); columns 2(-0); elements 4(-3)
Solving the presolved LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0     0.0000000000e+00 Ph1: 0(0) 0s
          2    -4.4000000000e+04 Pr: 0(0) 0s
Solving the original LP from the solution after postsolve
Model status        : Optimal
Simplex   iterations: 2
Objective value     :  4.4000000000e+04
Relative P-D gap    :  0.0000000000e+00
HiGHS run time      :          0.00

Status do solver: ok
Condição de terminação: optimal
Valor da Função Objetivo: R$ 44.000,00


In [272]:
for e in model.Empresas:
    valor = pyo.value(model.x[e])
    print(f"Investimento na empresa {e}: R$ {valor:,.2f}".replace(",", "X").replace(".", ",").replace("X", "."))



Investimento na empresa D: R$ 200.000,00
Investimento na empresa N: R$ 100.000,00


## EX15

In [273]:
# Criação do modelo
model = pyo.ConcreteModel()

In [274]:

# Conjuntos
# Tipos de aeronaves
model.Aeronaves = pyo.Set(initialize=["A1", "A2", "A3"])
# Rotas aéreas
model.Rotas = pyo.Set(initialize=["R1", "R2", "R3", "R4"])

In [275]:
# Capacidade de passageiros por tipo de aeronave
capacidade = {"A1": 50, "A2": 30, "A3": 20}
model.capacidade = pyo.Param(model.Aeronaves, initialize=capacidade)

# Número de aeronaves disponíveis por tipo
aeronaves_disponiveis = {"A1": 5, "A2": 8, "A3": 10}
model.aeronaves_disp = pyo.Param(model.Aeronaves, initialize=aeronaves_disponiveis)

# Número máximo de viagens por aeronave em cada rota
viagens_max = {
    ("A1", "R1"): 3, ("A1", "R2"): 2, ("A1", "R3"): 2, ("A1", "R4"): 1,
    ("A2", "R1"): 4, ("A2", "R2"): 3, ("A2", "R3"): 3, ("A2", "R4"): 2,
    ("A3", "R1"): 5, ("A3", "R2"): 5, ("A3", "R3"): 4, ("A3", "R4"): 2
}
model.viagens_max = pyo.Param(model.Aeronaves, model.Rotas, initialize=viagens_max)

# Demanda de passageiros por rota
demanda = {"R1": 100, "R2": 200, "R3": 90, "R4": 120}
model.demanda = pyo.Param(model.Rotas, initialize=demanda)

# Custos operacionais por viagem
custo_operacional = {
    ("A1", "R1"): 1000, ("A1", "R2"): 1100, ("A1", "R3"): 1200, ("A1", "R4"): 1500,
    ("A2", "R1"): 800,  ("A2", "R2"): 900,  ("A2", "R3"): 1000, ("A2", "R4"): 1000,
    ("A3", "R1"): 600,  ("A3", "R2"): 800,  ("A3", "R3"): 800,  ("A3", "R4"): 900
}
model.custo_viagem = pyo.Param(model.Aeronaves, model.Rotas, initialize=custo_operacional)


In [276]:

# ------------------------------
# Variáveis de Decisão
# ------------------------------

model.x = pyo.Var(model.Aeronaves, model.Rotas, domain=pyo.NonNegativeIntegers)


In [277]:

# ------------------------------
# Função Objetivo
# ------------------------------
def fo(model):
    return sum(model.x[a, r] * model.custo_viagem[a, r] for a in model.Aeronaves for r in model.Rotas)


model.objetivo = pyo.Objective(rule=fo, sense=pyo.maximize)


In [278]:
# ------------------------------
# Restrições
# ------------------------------

# 1. Atender à demanda de passageiros em cada rota
def demanda_rule(model, r):
    return sum(model.x[a, r] * model.capacidade[a] for a in model.Aeronaves) >= model.demanda[r]

model.demanda_atendida = pyo.Constraint(model.Rotas, rule=demanda_rule)

# 2. Não ultrapassar o número de aviões disponíveis
def limite_aeronaves_rule(model, a):
    return sum(model.x[a, r] for r in model.Rotas) <= model.aeronaves_disp[a]

model.limite_aeronaves = pyo.Constraint(model.Aeronaves, rule=limite_aeronaves_rule)

# 3. Respeitar o limite de viagens por aeronave em cada rota
def limite_viagens_rule(model, a, r):
    return model.x[a, r] <= model.viagens_max[a, r]
#
model.limite_viagens = pyo.Constraint(model.Aeronaves, model.Rotas, rule=limite_viagens_rule)



In [279]:
# ------------------------------
# Escrita do Modelo em Arquivo
# ------------------------------
model.write("ex15.lp", io_options={"symbolic_solver_labels": True})

('ex15.lp', 2676204125296)

In [280]:
# ------------------------------
# Resolução
# ------------------------------
solver = pyo.SolverFactory("appsi_highs")
results = solver.solve(model, tee=True)

# Exibindo resultados
print("\nStatus do solver:", results.solver.status)
print("Condição de terminação:", results.solver.termination_condition)
valor_obj = pyo.value(model.objetivo)
formatted = f"{valor_obj:,.2f}"
formatted = formatted.replace(",", "X").replace(".", ",").replace("X", ".")
print("Valor da Função Objetivo: R$", formatted)

MIP  has 19 rows; 12 cols; 36 nonzeros; 12 integer variables (0 binary)
Coefficient ranges:
  Matrix [1e+00, 5e+01]
  Cost   [6e+02, 2e+03]
  Bound  [0e+00, 0e+00]
  RHS    [1e+00, 2e+02]
Presolving model
7 rows, 12 cols, 24 nonzeros  0s
7 rows, 11 cols, 22 nonzeros  0s
Objective function is integral with scale 0.01

Solving MIP model with:
   7 rows
   11 cols (3 binary, 8 integer, 0 implied int., 0 continuous)
   22 nonzeros

Src: B => Branching; C => Central rounding; F => Feasibility pump; H => Heuristic; L => Sub-MIP;
     P => Empty MIP; R => Randomized rounding; S => Solve LP; T => Evaluate node; U => Unbounded;
     z => Trivial zero; l => Trivial lower; u => Trivial upper; p => Trivial point; X => User solution

        Nodes      |    B&B Tree     |            Objective Bounds              |  Dynamic Constraints |       Work      
Src  Proc. InQueue |  Leaves   Expl. | BestBound       BestSol              Gap |   Cuts   InLp Confl. | LpIters     Time

         0       0      

In [281]:

for a in model.Aeronaves:
    for r in model.Rotas:
        valor = pyo.value(model.x[a, r])
        if valor > 0:  # Exibir apenas valores positivos
            print(f"Número de viagens do avião {a} na rota {r}: {valor}")

Número de viagens do avião A1 na rota R1: 2.0
Número de viagens do avião A1 na rota R2: 1.0
Número de viagens do avião A1 na rota R3: 0.9999999999999998
Número de viagens do avião A1 na rota R4: 1.0
Número de viagens do avião A2 na rota R2: 3.0
Número de viagens do avião A2 na rota R3: 3.0
Número de viagens do avião A2 na rota R4: 2.0
Número de viagens do avião A3 na rota R2: 4.0
Número de viagens do avião A3 na rota R3: 4.0
Número de viagens do avião A3 na rota R4: 2.0


## Ex16

In [282]:
# Criação do modelo
model = pyo.ConcreteModel()

In [283]:

# Conjuntos
model.Cargas = pyo.Set(initialize=["A", "B", "C"])

# Compartimentos do navio
model.Compartimentos = pyo.Set(initialize=["Proa", "Centro", "Popa"])

In [284]:
# Lucro por tonelada de carga
lucro = {"A": 6000, "B": 8000, "C": 5000}
model.p_lucro = pyo.Param(model.Cargas, initialize=lucro)

# Capacidade máxima de peso em cada compartimento
capacidade_peso = {"Proa": 2000, "Centro": 3000, "Popa": 1500}
model.cap_peso = pyo.Param(model.Compartimentos, initialize=capacidade_peso)

# Capacidade máxima de volume em cada compartimento
capacidade_volume = {"Proa": 30000, "Centro": 40000, "Popa": 20000}
model.cap_volume = pyo.Param(model.Compartimentos, initialize=capacidade_volume)

# Volume específico por tonelada de carga
volume_especifico = {"A": 60, "B": 50, "C": 25}
model.v_especifico = pyo.Param(model.Cargas, initialize=volume_especifico)

# Disponibilidade máxima de carga
disponibilidade = {"A": 6000, "B": 4000, "C": 2000}
model.disp_carga = pyo.Param(model.Cargas, initialize=disponibilidade)

In [285]:

# ------------------------------
# Variáveis de Decisão
# ------------------------------

model.x = pyo.Var(model.Cargas, model.Compartimentos, domain=pyo.NonNegativeReals)


In [286]:

# ------------------------------
# Função Objetivo
# ------------------------------
def fo(model):
    return sum(model.x[i, j] * model.p_lucro[i] for i in model.Cargas for j in model.Compartimentos)

model.objetivo = pyo.Objective(rule=fo, sense=pyo.maximize)


In [287]:
# ------------------------------
# Restrições
# ------------------------------

# 1. Restrição de capacidade de peso por compartimento
def capacidade_peso_rule(model, j):
    return sum(model.x[i, j] for i in model.Cargas) <= model.cap_peso[j]

model.capacidade_peso = pyo.Constraint(model.Compartimentos, rule=capacidade_peso_rule)

# 2. Restrição de capacidade de volume por compartimento
def capacidade_volume_rule(model, j):
    return sum(model.x[i, j] * model.v_especifico[i] for i in model.Cargas) <= model.cap_volume[j]

model.capacidade_volume = pyo.Constraint(model.Compartimentos, rule=capacidade_volume_rule)

# 3. Restrição de disponibilidade de carga
def disponibilidade_rule(model, i):
    return sum(model.x[i, j] for j in model.Compartimentos) <= model.disp_carga[i]

model.disponibilidade = pyo.Constraint(model.Cargas, rule=disponibilidade_rule)

# 4. Restrição de equilíbrio de peso entre os compartimentos
def equilibrio_peso_rule_1(model):
    peso_proporcao = sum(model.x[i, "Proa"] for i in model.Cargas) / model.cap_peso["Proa"]
    peso_centro = sum(model.x[i, "Centro"] for i in model.Cargas) / model.cap_peso["Centro"]
    return peso_proporcao == peso_centro #== peso_popa
def equilibrio_peso_rule_2(model):
    peso_centro = sum(model.x[i, "Centro"] for i in model.Cargas) / model.cap_peso["Centro"]
    peso_popa = sum(model.x[i, "Popa"] for i in model.Cargas) / model.cap_peso["Popa"]
    return peso_centro == peso_popa
def equilibrio_peso_rule_3(model):
    peso_proporcao = sum(model.x[i, "Proa"] for i in model.Cargas) / model.cap_peso["Proa"]
    peso_popa = sum(model.x[i, "Popa"] for i in model.Cargas) / model.cap_peso["Popa"]
    return  peso_popa == peso_proporcao
model.equilibrio_peso = pyo.Constraint(rule=equilibrio_peso_rule_1)
model.equilibrio_peso_2 = pyo.Constraint(rule=equilibrio_peso_rule_2)
model.equilibrio_peso_3 = pyo.Constraint(rule=equilibrio_peso_rule_3)



In [288]:
# ------------------------------
# Escrita do Modelo em Arquivo
# ------------------------------
model.write("ex16.lp", io_options={"symbolic_solver_labels": True})

('ex16.lp', 2676203526656)

In [289]:
# ------------------------------
# Resolução
# ------------------------------
solver = pyo.SolverFactory("appsi_highs")
results = solver.solve(model, tee=True)

# Exibindo resultados
print("\nStatus do solver:", results.solver.status)
print("Condição de terminação:", results.solver.termination_condition)
valor_obj = pyo.value(model.objetivo)
formatted = f"{valor_obj:,.2f}"
formatted = formatted.replace(",", "X").replace(".", ",").replace("X", ".")
print("Valor da Função Objetivo: R$", formatted)

LP   has 12 rows; 9 cols; 45 nonzeros
Coefficient ranges:
  Matrix [3e-04, 6e+01]
  Cost   [5e+03, 8e+03]
  Bound  [0e+00, 0e+00]
  RHS    [2e+03, 4e+04]
Presolving model
10 rows, 9 cols, 39 nonzeros  0s
Dependent equations search running on 3 equations with time limit of 1000.00s
Dependent equations search removed 1 rows and 6 nonzeros in 0.00s (limit = 1000.00s)
9 rows, 9 cols, 33 nonzeros  0s
Presolve : Reductions: rows 9(-3); columns 9(-0); elements 33(-12)
Solving the presolved LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0    -4.1599492483e+05 Ph1: 8(22.9519); Du: 9(415995) 0s
          8    -1.6400000000e+07 Pr: 0(0) 0s
Solving the original LP from the solution after postsolve
Model status        : Optimal
Simplex   iterations: 8
Objective value     :  1.6400000000e+07
Relative P-D gap    :  0.0000000000e+00
HiGHS run time      :          0.00

Status do solver: ok
Condição de terminação: optimal
Valor da Função Ob

In [290]:

for i in model.Cargas:
    for j in model.Compartimentos:
        valor = pyo.value(model.x[i, j])
        print(f"Carga {i} no compartimento {j}: {valor:.2f} toneladas")



Carga A no compartimento Proa: 0.00 toneladas
Carga A no compartimento Centro: 0.00 toneladas
Carga A no compartimento Popa: 0.00 toneladas
Carga B no compartimento Proa: 338.46 toneladas
Carga B no compartimento Centro: 307.69 toneladas
Carga B no compartimento Popa: 153.85 toneladas
Carga C no compartimento Proa: 523.08 toneladas
Carga C no compartimento Centro: 984.62 toneladas
Carga C no compartimento Popa: 492.31 toneladas


## EX17

In [291]:
# Criação do modelo
model = pyo.ConcreteModel()

In [293]:
# Conjuntos
model.Matrizes = pyo.Set(initialize=["1", "2", "3", "4"])

In [294]:
# Custo por chapa cortada utilizando cada matriz
custo = {"1": 1, "2": 2, "3": 3, "4": 2}
model.custo_chapa = pyo.Param(model.Matrizes, initialize=custo)

# Produção de panelas médias por matriz
prod_medias = {"1": 8, "2": 4, "3": 2, "4": 0}
model.prod_medias = pyo.Param(model.Matrizes, initialize=prod_medias)

# Produção de panelas grandes por matriz
prod_grandes = {"1": 0, "2": 1, "3": 2, "4": 3}
model.prod_grandes = pyo.Param(model.Matrizes, initialize=prod_grandes)

# Demanda mínima de produção diária
model.demanda_medias = pyo.Param(initialize=500)
model.demanda_grandes = pyo.Param(initialize=350)

In [295]:

# ------------------------------
# Variáveis de Decisão
# ------------------------------

model.x = pyo.Var(model.Matrizes, domain=pyo.NonNegativeIntegers)

In [296]:

# ------------------------------
# Função Objetivo
# ------------------------------
def fo(model):
    return sum(model.x[j] * model.custo_chapa[j] for j in model.Matrizes)

model.objetivo = pyo.Objective(rule=fo, sense=pyo.minimize)


In [297]:
# ------------------------------
# Restrições
# ------------------------------

# 1. Produção mínima de panelas médias
def producao_medias_rule(model):
    return sum(model.x[j] * model.prod_medias[j] for j in model.Matrizes) >= model.demanda_medias

model.producao_medias = pyo.Constraint(rule=producao_medias_rule)

# 2. Produção mínima de panelas grandes
def producao_grandes_rule(model):
    return sum(model.x[j] * model.prod_grandes[j] for j in model.Matrizes) >= model.demanda_grandes

model.producao_grandes = pyo.Constraint(rule=producao_grandes_rule)



In [298]:
# ------------------------------
# Escrita do Modelo em Arquivo
# ------------------------------
model.write("ex17.lp", io_options={"symbolic_solver_labels": True})

('ex17.lp', 2676203525120)

In [299]:
# ------------------------------
# Resolução
# ------------------------------
solver = pyo.SolverFactory("appsi_highs")
results = solver.solve(model, tee=True)

# Exibindo resultados
print("\nStatus do solver:", results.solver.status)
print("Condição de terminação:", results.solver.termination_condition)
valor_obj = pyo.value(model.objetivo)
formatted = f"{valor_obj:,.2f}"
formatted = formatted.replace(",", "X").replace(".", ",").replace("X", ".")
print("Valor da Função Objetivo: R$", formatted)

MIP  has 2 rows; 4 cols; 6 nonzeros; 4 integer variables (0 binary)
Coefficient ranges:
  Matrix [1e+00, 8e+00]
  Cost   [1e+00, 3e+00]
  Bound  [0e+00, 0e+00]
  RHS    [4e+02, 5e+02]
Presolving model
2 rows, 4 cols, 6 nonzeros  0s
2 rows, 4 cols, 6 nonzeros  0s
Objective function is integral with scale 1

Solving MIP model with:
   2 rows
   4 cols (0 binary, 4 integer, 0 implied int., 0 continuous)
   6 nonzeros

Src: B => Branching; C => Central rounding; F => Feasibility pump; H => Heuristic; L => Sub-MIP;
     P => Empty MIP; R => Randomized rounding; S => Solve LP; T => Evaluate node; U => Unbounded;
     z => Trivial zero; l => Trivial lower; u => Trivial upper; p => Trivial point; X => User solution

        Nodes      |    B&B Tree     |            Objective Bounds              |  Dynamic Constraints |       Work      
Src  Proc. InQueue |  Leaves   Expl. | BestBound       BestSol              Gap |   Cuts   InLp Confl. | LpIters     Time

         0       0         0   0.00% 

In [300]:

for j in model.Matrizes:
    valor = pyo.value(model.x[j])
    print(f"Matriz {j}: {valor:.2f} chapas")

Matriz 1: 63.00 chapas
Matriz 2: 0.00 chapas
Matriz 3: 0.00 chapas
Matriz 4: 117.00 chapas


## Ex18

In [301]:
# Criação do modelo
model = pyo.ConcreteModel()

In [302]:

# Conjuntos
model.Matrizes = pyo.Set(initialize=[1, 2, 3, 4])


In [303]:
# Coeficientes de produção de extremidades por matriz
coef_extremidades = {1: 3, 2: 7, 3: 5, 4: 9}
model.coef_ext = pyo.Param(model.Matrizes, initialize=coef_extremidades)

# Coeficientes de produção de corpos por matriz
coef_corpos = {1: 3, 2: 1, 3: 4, 4: 3}
model.coef_corpo = pyo.Param(model.Matrizes, initialize=coef_corpos)

# Demanda mínima
demanda_extremidades = 10000
demanda_corpos = 5000

In [304]:

# ------------------------------
# Variáveis de Decisão
# ------------------------------

model.x = pyo.Var(model.Matrizes, domain=pyo.NonNegativeIntegers)

In [305]:

# ------------------------------
# Função Objetivo
# ------------------------------
def fo(model):
    return sum(model.x[j] for j in model.Matrizes)

model.objetivo = pyo.Objective(rule=fo, sense=pyo.minimize)


In [306]:
# ------------------------------
# Restrições
# ------------------------------

# 1. Produção mínima de extremidades
def restricao_extremidades(model):
    return sum(model.coef_ext[j] * model.x[j] for j in model.Matrizes) >= demanda_extremidades

model.restricao_extremidades = pyo.Constraint(rule=restricao_extremidades)

# 2. Produção mínima de corpos
def restricao_corpos(model):
    return sum(model.coef_corpo[j] * model.x[j] for j in model.Matrizes) >= demanda_corpos

model.restricao_corpos = pyo.Constraint(rule=restricao_corpos)

In [307]:
# ------------------------------
# Escrita do Modelo em Arquivo
# ------------------------------
model.write("ex18.lp", io_options={"symbolic_solver_labels": True})

('ex18.lp', 2676203526176)

In [308]:
# ------------------------------
# Resolução
# ------------------------------
solver = pyo.SolverFactory("appsi_highs")
results = solver.solve(model, tee=True)

# Exibindo resultados
print("\nStatus do solver:", results.solver.status)
print("Condição de terminação:", results.solver.termination_condition)
valor_obj = pyo.value(model.objetivo)
formatted = f"{valor_obj:,.2f}"
formatted = formatted.replace(",", "X").replace(".", ",").replace("X", ".")
print("Valor da Função Objetivo: ", formatted)

MIP  has 2 rows; 4 cols; 8 nonzeros; 4 integer variables (0 binary)
Coefficient ranges:
  Matrix [1e+00, 9e+00]
  Cost   [1e+00, 1e+00]
  Bound  [0e+00, 0e+00]
  RHS    [5e+03, 1e+04]
Presolving model
2 rows, 4 cols, 8 nonzeros  0s
2 rows, 4 cols, 8 nonzeros  0s
Objective function is integral with scale 1

Solving MIP model with:
   2 rows
   4 cols (0 binary, 4 integer, 0 implied int., 0 continuous)
   8 nonzeros

Src: B => Branching; C => Central rounding; F => Feasibility pump; H => Heuristic; L => Sub-MIP;
     P => Empty MIP; R => Randomized rounding; S => Solve LP; T => Evaluate node; U => Unbounded;
     z => Trivial zero; l => Trivial lower; u => Trivial upper; p => Trivial point; X => User solution

        Nodes      |    B&B Tree     |            Objective Bounds              |  Dynamic Constraints |       Work      
Src  Proc. InQueue |  Leaves   Expl. | BestBound       BestSol              Gap |   Cuts   InLp Confl. | LpIters     Time

         0       0         0   0.00% 

In [309]:

for j in model.Matrizes:
    valor = pyo.value(model.x[j])
    print(f"Matriz {j}: {valor:.2f} chapas")

Matriz 1: 0.00 chapas
Matriz 2: 0.00 chapas
Matriz 3: 714.00 chapas
Matriz 4: 715.00 chapas


## Ex19

In [310]:
# Criação do modelo
model = pyo.ConcreteModel()

In [311]:

# Conjuntos
model.T = pyo.RangeSet(1, 10) 
model.I = pyo.RangeSet(1, 4)   


In [312]:
# Custo de alistamento por tempo de serviço
custo_alistamento = {1: 10, 2: 18, 3: 25, 4: 31}
model.custo = pyo.Param(model.I, initialize=custo_alistamento)

# Demanda de reservistas por ano
demanda_reservistas = {1: 10200, 2: 11800, 3: 10500, 4: 12300, 5: 15000,
                       6: 14500, 7: 14000, 8: 16100, 9: 16400, 10: 16500}
model.demanda = pyo.Param(model.T, initialize=demanda_reservistas)

# Capacidade máxima de recrutamento por ano
capacidade_recrutamento = 13000

In [313]:

# ------------------------------
# Variáveis de Decisão
# ------------------------------

model.x = pyo.Var(model.T, model.I, domain=pyo.NonNegativeIntegers)

In [314]:

# ------------------------------
# Função Objetivo
# ------------------------------
def fo(model):
    return sum(model.x[t, i] * model.custo[i] for t in model.T for i in model.I)

model.objetivo = pyo.Objective(rule=fo, sense=pyo.minimize)


In [315]:
# ------------------------------
# Restrições
# ------------------------------

# 1. Exigência mínima de reservistas em cada ano
def restricao_demanda(model, t):
    a = 0
    if t >= 1:
        a += model.x[t, 1]
    if t-1 >= 1:
        a += model.x[t-1, 2]
    if t-2 >= 1:
        a += model.x[t-2, 3]
    if t-3 >= 1:
        a += model.x[t-3, 4]
    return a >= model.demanda[t]

model.restricao_demanda = pyo.Constraint(model.T, rule=restricao_demanda)

# 2. Limite máximo de recrutamento por ano
def restricao_max_recrutamento(model, t):
    return sum(model.x[t, i] for i in model.I) <= capacidade_recrutamento

model.restricao_max_recrutamento = pyo.Constraint(model.T, rule=restricao_max_recrutamento)


In [316]:
# ------------------------------
# Escrita do Modelo em Arquivo
# ------------------------------
model.write("ex19.lp", io_options={"symbolic_solver_labels": True})

('ex19.lp', 2676203576624)

In [317]:
# ------------------------------
# Resolução
# ------------------------------
solver = pyo.SolverFactory("appsi_highs")
results = solver.solve(model, tee=True)

# Exibindo resultados
print("\nStatus do solver:", results.solver.status)
print("Condição de terminação:", results.solver.termination_condition)
valor_obj = pyo.value(model.objetivo)
formatted = f"{valor_obj:,.2f}"
formatted = formatted.replace(",", "X").replace(".", ",").replace("X", ".")
print("Valor da Função Objetivo: R$", formatted)

MIP  has 20 rows; 40 cols; 74 nonzeros; 40 integer variables (0 binary)
Coefficient ranges:
  Matrix [1e+00, 1e+00]
  Cost   [1e+01, 3e+01]
  Bound  [0e+00, 0e+00]
  RHS    [1e+04, 2e+04]
Presolving model
19 rows, 33 cols, 66 nonzeros  0s
16 rows, 31 cols, 61 nonzeros  0s
Objective function is integral with scale 1

Solving MIP model with:
   16 rows
   31 cols (0 binary, 31 integer, 0 implied int., 0 continuous)
   61 nonzeros

Src: B => Branching; C => Central rounding; F => Feasibility pump; H => Heuristic; L => Sub-MIP;
     P => Empty MIP; R => Randomized rounding; S => Solve LP; T => Evaluate node; U => Unbounded;
     z => Trivial zero; l => Trivial lower; u => Trivial upper; p => Trivial point; X => User solution

        Nodes      |    B&B Tree     |            Objective Bounds              |  Dynamic Constraints |       Work      
Src  Proc. InQueue |  Leaves   Expl. | BestBound       BestSol              Gap |   Cuts   InLp Confl. | LpIters     Time

         0       0     

RuntimeError: A feasible solution was not found, so no solution can be loaded.Please set opt.config.load_solution=False and check results.termination_condition and results.best_feasible_objective before loading a solution.

MODELO INFACTIVEL

## Ex20

In [318]:
# Criação do modelo
model = pyo.ConcreteModel()

In [319]:

# Conjuntos
model.I = pyo.Set(initialize=["I", "II", "III", "IV", "V"])
model.T = pyo.RangeSet(0, 3) 


In [320]:
# Capital inicial disponível (R$ mil)
capital_inicial = 220

# Taxa de juros da poupança (10% ao ano)
taxa_juros = 1.1

# Fluxo de caixa dos projetos (R$ mil)
custo_inv = { "I":100, "II":150, "III":100, "IV":50, "V":70 }
retorno_in = { "I":170, "II":220, "III":150, "IV":80, "V":130 }
model.custo_inv = pyo.Param(model.I, initialize=custo_inv, default=0)
model.retorno_in = pyo.Param(model.I, initialize=retorno_in, default=0)


In [321]:

# ------------------------------
# Variáveis de Decisão
# ------------------------------

model.x = pyo.Var(model.I, domain=pyo.Binary)
model.S = pyo.Var(model.T, domain=pyo.NonNegativeReals)

In [322]:

# ------------------------------
# Função Objetivo
# ------------------------------
def fo(model):
    return model.S[3]*taxa_juros

model.objetivo = pyo.Objective(rule=fo, sense=pyo.maximize)


In [323]:
# ------------------------------
# Restrições
# ------------------------------
# 1. Restrição de orçamento inicial (ano 0)
def restricao_orcamento(model):

    return model.custo_inv['I']*model.x['I'] +\
          model.custo_inv['II']*model.x['II']+\
          model.custo_inv['IV']*model.x['IV'] \
            + model.S[0] == capital_inicial

model.restricao_orcamento = pyo.Constraint(rule=restricao_orcamento)

# 2. Evolução do saldo na poupança 

def evolucao_saldo_ano_1(model):
    return model.S[1] + model.x['III'] * model.custo_inv['III'] + model.x['V']*model.custo_inv['V'] == \
    model.S[0]*taxa_juros+model.x['IV']*model.retorno_in['IV'] 


model.evolucao_saldo_ano_1 = pyo.Constraint(rule=evolucao_saldo_ano_1)

def evolucao_saldo_ano_2(model):
    return model.S[2] == \
    model.S[1]*taxa_juros+model.x['I']*model.retorno_in['I'] 

model.evolucao_saldo_ano_2 = pyo.Constraint(rule=evolucao_saldo_ano_2)

def evolucao_saldo_ano_3(model):
    return model.S[3] == \
    model.S[2]*taxa_juros+model.x['II']*model.retorno_in['II'] +model.x['III']*model.retorno_in['III'] + model.x['V']*model.retorno_in['V']

model.evolucao_saldo_ano_3 = pyo.Constraint(rule=evolucao_saldo_ano_3)
# 3. Mutual exclusividade entre os projetos I e V
def exclusividade(model):
    return model.x["I"] + model.x["V"] <= 1

model.exclusividade = pyo.Constraint(rule=exclusividade)


In [324]:
# ------------------------------
# Escrita do Modelo em Arquivo
# ------------------------------
model.write("ex20.lp", io_options={"symbolic_solver_labels": True})

('ex20.lp', 2676230358208)

In [325]:
# ------------------------------
# Resolução
# ------------------------------
solver = pyo.SolverFactory("appsi_highs")
results = solver.solve(model, tee=True)

# Exibindo resultados
print("\nStatus do solver:", results.solver.status)
print("Condição de terminação:", results.solver.termination_condition)

MIP  has 5 rows; 9 cols; 19 nonzeros; 5 integer variables (5 binary)
Coefficient ranges:
  Matrix [1e+00, 2e+02]
  Cost   [1e+00, 1e+00]
  Bound  [1e+00, 1e+00]
  RHS    [1e+00, 2e+02]
Presolving model
4 rows, 8 cols, 14 nonzeros  0s
3 rows, 7 cols, 11 nonzeros  0s
Objective function is integral with scale 90.9091

Solving MIP model with:
   3 rows
   7 cols (5 binary, 0 integer, 2 implied int., 0 continuous)
   11 nonzeros

Src: B => Branching; C => Central rounding; F => Feasibility pump; H => Heuristic; L => Sub-MIP;
     P => Empty MIP; R => Randomized rounding; S => Solve LP; T => Evaluate node; U => Unbounded;
     z => Trivial zero; l => Trivial lower; u => Trivial upper; p => Trivial point; X => User solution

        Nodes      |    B&B Tree     |            Objective Bounds              |  Dynamic Constraints |       Work      
Src  Proc. InQueue |  Leaves   Expl. | BestBound       BestSol              Gap |   Cuts   InLp Confl. | LpIters     Time

         0       0         

In [326]:
print("Projetos escolhidos :")
for i in model.I:
    if model.x[i].value > 0.5:
        print(f"Projeto {i}")

print("\nSaldo na poupança ao final de cada ano:")
for t in model.T:
    print(f" ano {t}: R$ {model.S[t].value * 1000:.2f}")

print("\nSaldo final na poupança ao fim do ano 3:")
print(f" R$ {model.S[3].value * 1000:.2f}")

Projetos escolhidos :
Projeto I
Projeto III
Projeto IV

Saldo na poupança ao final de cada ano:
 ano 0: R$ 70000.00
 ano 1: R$ 57000.00
 ano 2: R$ 232700.00
 ano 3: R$ 405970.00

Saldo final na poupança ao fim do ano 3:
 R$ 405970.00


## Ex21

In [327]:
# Criação do modelo
model = pyo.ConcreteModel()

In [328]:

# Conjuntos
model.I = pyo.Set(initialize=["I", "II", "III", "IV", "V","VI"])



In [329]:
# Valor presente líquido (VPL) de cada projeto (R$ milhões)
vpl = {"I": 8, "II": 7, "III": 5, "IV": 6, "V": 4, "VI": 9}
model.vpl = pyo.Param(model.I, initialize=vpl)

# Capital requerido para cada projeto (R$ milhões)
custo = {"I": 6, "II": 5, "III": 3, "IV": 4, "V": 3, "VI": 7}
model.custo = pyo.Param(model.I, initialize=custo)

# Orçamento disponível (R$ milhões)
budget = 20

In [330]:

# ------------------------------
# Variáveis de Decisão
# ------------------------------

model.x =  pyo.Var(model.I, domain=pyo.Binary)

In [331]:

# ------------------------------
# Função Objetivo
# ------------------------------
def fo(model):
    return sum(model.vpl[i] * model.x[i] for i in model.I)

model.objetivo = pyo.Objective(rule=fo, sense=pyo.maximize)


In [332]:
# ------------------------------
# Restrições
# ------------------------------
# 1. Restrição de orçamento
def restricao_orcamento(model):
    return sum(model.custo[i] * model.x[i] for i in model.I) == budget

model.restricao_orcamento = pyo.Constraint(rule=restricao_orcamento)

# 2. Mutual exclusividade entre os projetos I e II
def exclusividade_I_II(model):
    return model.x["I"] + model.x["II"] <= 1

model.exclusividade_I_II = pyo.Constraint(rule=exclusividade_I_II)

# 3. Mutual exclusividade entre os projetos IV e V
def exclusividade_IV_V(model):
    return model.x["IV"] + model.x["V"] <= 1

model.exclusividade_IV_V = pyo.Constraint(rule=exclusividade_IV_V)

# 4. Dependência do projeto III em relação ao projeto I
def dependencia_III_I(model):
    return model.x["III"] <= model.x["I"]

model.dependencia_III_I = pyo.Constraint(rule=dependencia_III_I)



In [333]:
# ------------------------------
# Escrita do Modelo em Arquivo
# ------------------------------
model.write("ex21.lp", io_options={"symbolic_solver_labels": True})

('ex21.lp', 2676203576000)

In [334]:
# ------------------------------
# Resolução
# ------------------------------
solver = pyo.SolverFactory("appsi_highs")
results = solver.solve(model, tee=True)

# Exibindo resultados
print("\nStatus do solver:", results.solver.status)
print("Condição de terminação:", results.solver.termination_condition)


MIP  has 4 rows; 6 cols; 12 nonzeros; 6 integer variables (6 binary)
Coefficient ranges:
  Matrix [1e+00, 7e+00]
  Cost   [4e+00, 9e+00]
  Bound  [1e+00, 1e+00]
  RHS    [1e+00, 2e+01]
Presolving model
4 rows, 6 cols, 12 nonzeros  0s
4 rows, 6 cols, 12 nonzeros  0s
Objective function is integral with scale 1

Solving MIP model with:
   4 rows
   6 cols (6 binary, 0 integer, 0 implied int., 0 continuous)
   12 nonzeros

Src: B => Branching; C => Central rounding; F => Feasibility pump; H => Heuristic; L => Sub-MIP;
     P => Empty MIP; R => Randomized rounding; S => Solve LP; T => Evaluate node; U => Unbounded;
     z => Trivial zero; l => Trivial lower; u => Trivial upper; p => Trivial point; X => User solution

        Nodes      |    B&B Tree     |            Objective Bounds              |  Dynamic Constraints |       Work      
Src  Proc. InQueue |  Leaves   Expl. | BestBound       BestSol              Gap |   Cuts   InLp Confl. | LpIters     Time

         0       0         0   0.

In [335]:
print("\nProjetos escolhidos para investimento:")
for i in model.I:
    if model.x[i].value > 0.5:
        print(f"Projeto {i}")

print("\nVPL total maximizado:")
print(f"R$ {model.objetivo()} milhões")


Projetos escolhidos para investimento:
Projeto I
Projeto III
Projeto IV
Projeto VI

VPL total maximizado:
R$ 28.0 milhões


## Ex22

In [336]:
# Criação do modelo
model = pyo.ConcreteModel()

In [337]:

# Conjuntos
model.M = pyo.Set(initialize=[0,1,2,3,4,5])



In [338]:
H_m = {0:8000, 1:9000, 2:7000, 3:10000, 4:9000, 5:11000}  # Demanda de horas de voo por mês
model.H_m = pyo.Param(model.M, initialize=H_m)
Cx = 1500  # Salário de uma aeromoça regular por mês
model.Cx = pyo.Param(initialize=Cx)
Cy = 900  # Salário de uma aeromoça em treinamento por mês
model.Cy = pyo.Param(initialize=Cy)
HM = 150  # Horas de trabalho máximas por aeromoça regular por mês
model.HM = pyo.Param(initialize=HM)
HT = 100  # Horas de supervisão de uma aeromoça em treinamento
model.HT = pyo.Param(initialize=HT)
n_meses = len(H_m)  # Número de meses
model.n_meses = pyo.Param(initialize=n_meses)
x_ini = 60  # Aeromoças regulares no mês de janeiro
model.x_ini = pyo.Param(initialize=x_ini)

In [339]:

# ------------------------------
# Variáveis de Decisão
# ------------------------------

# Número de aeromoças experientes em cada mês
model.x = pyo.Var(model.M, domain=pyo.NonNegativeIntegers)

# Número de aeromoças em treinamento em cada mês
model.y = pyo.Var(model.M, domain=pyo.NonNegativeIntegers)

In [340]:

# ------------------------------
# Função Objetivo
# ------------------------------
def fo(model):
    return sum(model.Cx * model.x[m] + model.Cy * model.y[m] for m in model.M)

model.objetivo = pyo.Objective(rule=fo, sense=pyo.minimize)


In [341]:
# ------------------------------
# Restrições
# ------------------------------
# 1. Horas de Voo
def restricao_horas_voo(model, m):
    return model.HM * model.x[m] - model.HT * model.y[m] >= model.H_m[m]  

model.restricao_horas_voo = pyo.Constraint(model.M, rule=restricao_horas_voo)

# 2. Redução de aeromoças experientes ao longo dos meses
def reducao_inf(model, m):
    if m < n_meses-1:  
        return model.x[m + 1] >= 0.895 * model.x[m] + model.y[m]
    return pyo.Constraint.Skip

model.reducao_inf = pyo.Constraint(model.M, rule=reducao_inf)

def reducao_sup(model, m):
    if m < model.n_meses-1:  
        return model.x[m + 1] <= 0.91 * model.x[m] + model.y[m]
    return pyo.Constraint.Skip

model.reducao_sup = pyo.Constraint(model.M, rule=reducao_sup)

# 3. Número de aeromoças experientes no mês de janeiro
model.restricao_inicio = pyo.Constraint(expr=model.x[0] == model.x_ini)




In [342]:
# ------------------------------
# Escrita do Modelo em Arquivo
# ------------------------------
model.write("ex22.lp", io_options={"symbolic_solver_labels": True})

('ex22.lp', 2676204071568)

In [343]:
# ------------------------------
# Resolução
# ------------------------------
solver = pyo.SolverFactory("appsi_highs")
results = solver.solve(model, tee=True)

# Exibindo resultados
print("\nStatus do solver:", results.solver.status)
print("Condição de terminação:", results.solver.termination_condition)


MIP  has 17 rows; 12 cols; 43 nonzeros; 12 integer variables (0 binary)
Coefficient ranges:
  Matrix [9e-01, 2e+02]
  Cost   [9e+02, 2e+03]
  Bound  [0e+00, 0e+00]
  RHS    [6e+01, 1e+04]
Presolving model
16 rows, 10 cols, 38 nonzeros  0s
12 rows, 9 cols, 32 nonzeros  0s
12 rows, 9 cols, 32 nonzeros  0s
Objective function is integral with scale 0.00333333

Solving MIP model with:
   12 rows
   9 cols (0 binary, 9 integer, 0 implied int., 0 continuous)
   32 nonzeros

Src: B => Branching; C => Central rounding; F => Feasibility pump; H => Heuristic; L => Sub-MIP;
     P => Empty MIP; R => Randomized rounding; S => Solve LP; T => Evaluate node; U => Unbounded;
     z => Trivial zero; l => Trivial lower; u => Trivial upper; p => Trivial point; X => User solution

        Nodes      |    B&B Tree     |            Objective Bounds              |  Dynamic Constraints |       Work      
Src  Proc. InQueue |  Leaves   Expl. | BestBound       BestSol              Gap |   Cuts   InLp Confl. | Lp

In [344]:

print("\nNúmero de aeromoças experientes em cada mês:")
for m in model.M:
    print(f"Mês {m + 1}: {model.x[m].value:.0f}")

print("\nNúmero de aeromoças em treinamento em cada mês:")
for m in model.M:
    print(f"Mês {m + 1}: {model.y[m].value:.0f}")


Número de aeromoças experientes em cada mês:
Mês 1: 60
Mês 2: 62
Mês 3: 59
Mês 4: 71
Mês 5: 69
Mês 6: 74

Número de aeromoças em treinamento em cada mês:
Mês 1: 8
Mês 2: 3
Mês 3: 18
Mês 4: 5
Mês 5: 12
Mês 6: 0
